In [4]:
import os
import openai
from IPython.display import HTML, display
from google.colab.output import eval_js
from base64 import b64decode
from scipy.io.wavfile import read as wav_read
import io
import ffmpeg
import spacy
from googletrans import Translator

# Set the OpenAI API key (replace with your actual key)
os.environ["OPENAI_API_KEY"] = ""
openai.api_key = os.getenv("OPENAI_API_KEY")

# HTML/JavaScript for recording audio in Colab
AUDIO_HTML = """
<script>
var my_div = document.createElement('div');
var my_p = document.createElement('p');
my_p.innerHTML = "Click the button to start recording:";
my_div.appendChild(my_p);

var my_button = document.createElement('button');
my_button.innerHTML = "Record";
my_button.onclick = async function() {
  const stream = await navigator.mediaDevices.getUserMedia({ audio: true });
  const mediaRecorder = new MediaRecorder(stream);
  mediaRecorder.start();

  const audioChunks = [];
  mediaRecorder.addEventListener("dataavailable", event => {
    audioChunks.push(event.data);
  });

  mediaRecorder.addEventListener("stop", async () => {
    const audioBlob = new Blob(audioChunks);
    const reader = new FileReader();
    reader.readAsDataURL(audioBlob);
    reader.onloadend = () => {
      const base64data = reader.result;
      google.colab.kernel.invokeFunction('notebook.record_audio', [base64data], {});
    };
  });

  setTimeout(() => {
    mediaRecorder.stop();
  }, 5000);
};
my_div.appendChild(my_button);
document.body.appendChild(my_div);
</script>
"""

# Initialize SpaCy models
nlp_en = spacy.load("en_core_web_sm")

# Initialize Google Translator
translator = Translator()

def translate_text_openai(text, target_language="en"):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": f"Translate the following text to {target_language}: {text}"}
        ]
    )
    translated_text = response.choices[0].message.content.strip()
    return translated_text

def translate_audio_to_text(audio_data):
    response = openai.Audio.transcribe(
        model="whisper-1",
        file=io.BytesIO(audio_data),
        format="wav"
    )
    return response["text"]

def extract_locations(user_input, input_language="en"):
    if input_language != "en":
        user_input = translator.translate(user_input, dest="en").text

    # Define keywords that indicate a direction or route request
    direction_keywords = ["from", "to", "take", "route", "drive"]

    # Split user input into words
    words = user_input.lower().split()

    # Initialize variables to store start and end locations
    start_location = None
    end_location = None

    # Use English SpaCy model for extraction
    doc = nlp_en(user_input)

    for ent in doc.ents:
        if ent.label_ == "LOC":
            if start_location is None:
                start_location = ent.text.capitalize()  # Assuming location is in title case
            elif end_location is None:
                end_location = ent.text.capitalize()  # Assuming location is in title case

    # Fallback to keyword-based extraction for other languages or if no entities found
    if not start_location or not end_location:
        for i in range(len(words)):
            if words[i] in direction_keywords:
                if i + 1 < len(words):
                    if start_location is None:
                        start_location = words[i + 1].capitalize()  # Assuming location is in title case
                    elif end_location is None:
                        end_location = words[i + 1].capitalize()  # Assuming location is in title case

    return start_location, end_location

def generate_route_description(start_location, end_location):
    prompt = f"Provide a detailed driving route from {start_location} to {end_location}. Include major turns, landmarks, and distances."

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ]
    )

    return response.choices[0].message.content.strip()

def main():
    print("Route Description Chatbot. Type 'exit' to quit.")
    while True:
        input_type = input("Enter 'text' for text input or 'audio' for audio input: ").strip().lower()

        if input_type == 'exit':
            print('Exiting')
            break

        if input_type not in ['text', 'audio']:
            print("Invalid input type. Please enter 'text' or 'audio'.")
            continue

        if input_type == 'audio':
            display(HTML(AUDIO_HTML))
            data = eval_js("base64data")
            binary = b64decode(data.split(',')[1])

            process = (
                ffmpeg.input('pipe:0')
                .output('pipe:1', format='wav')
                .run_async(pipe_stdin=True, pipe_stdout=True, pipe_stderr=True, quiet=True, overwrite_output=True, input=binary)
            )
            output, _ = process.communicate()

            riff_chunk_size = len(output) - 8
            q = riff_chunk_size
            b = []
            for i in range(4):
                q, r = divmod(q, 256)
                b.append(r)

            riff = output[:4] + bytes(b) + output[8:]

            sr, audio = wav_read(io.BytesIO(riff))

            audio_data = translate_audio_to_text(audio.tobytes())
            print("Translated Audio:", audio_data)

            # Detect language of audio input
            input_language = "hi" if any('\u0900' <= char <= '\u097f' for char in audio_data) else "en"

            start_location, end_location = extract_locations(audio_data, input_language)
            if not start_location or not end_location:
                print("Could not extract start and end locations from input. Please try again.")
                continue

            route_description = generate_route_description(start_location, end_location)
            print("Route Description:", route_description)

        else:
            user_input = input("Enter your request: ")

            language_code = None
            if ":" in user_input:
                parts = user_input.split(":")
                user_input = parts[0].strip()
                language_code = parts[1].strip().lower()

            start_location, end_location = extract_locations(user_input, language_code)
            if not start_location or not end_location:
                print("Could not extract start and end locations from input. Please try again.")
                continue

            route_description = generate_route_description(start_location, end_location)
            print("Route Description:", route_description)

if __name__ == "__main__":
    main()


ImportError: cannot import name 'BaseTransport' from 'httpx' (/usr/local/lib/python3.10/dist-packages/httpx/__init__.py)

In [1]:
!pip install openai transformers spacy pydub
!python -m spacy download en_core_web_sm
!pip install ffmpeg-python


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 34.9 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [3]:
import os
import openai
from IPython.display import HTML, display
from google.colab.output import eval_js
from base64 import b64decode
from scipy.io.wavfile import read as wav_read
import io
import ffmpeg
import spacy
from transformers import pipeline

# Set the OpenAI API key (replace with your actual key)
os.environ["OPENAI_API_KEY"] = "
"
openai.api_key = os.getenv("OPENAI_API_KEY")

# HTML/JavaScript for recording audio in Colab
AUDIO_HTML = """
<script>
var my_div = document.createElement('div');
var my_p = document.createElement('p');
my_p.innerHTML = "Click the button to start recording:";
my_div.appendChild(my_p);

var my_button = document.createElement('button');
my_button.innerHTML = "Record";
my_button.onclick = async function() {
  const stream = await navigator.mediaDevices.getUserMedia({ audio: true });
  const mediaRecorder = new MediaRecorder(stream);
  mediaRecorder.start();

  const audioChunks = [];
  mediaRecorder.addEventListener("dataavailable", event => {
    audioChunks.push(event.data);
  });

  mediaRecorder.addEventListener("stop", async () => {
    const audioBlob = new Blob(audioChunks);
    const reader = new FileReader();
    reader.readAsDataURL(audioBlob);
    reader.onloadend = () => {
      const base64data = reader.result;
      google.colab.kernel.invokeFunction('notebook.record_audio', [base64data], {});
    };
  });

  setTimeout(() => {
    mediaRecorder.stop();
  }, 5000);
};
my_div.appendChild(my_button);
document.body.appendChild(my_div);
</script>
"""

# Initialize SpaCy models
nlp_en = spacy.load("en_core_web_sm")

# Initialize Hugging Face translation pipeline
translator = pipeline("translation", model="Helsinki-NLP/opus-mt-hi-en")

def translate_text_openai(text, target_language="en"):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": f"Translate the following text to {target_language}: {text}"}
        ]
    )
    translated_text = response.choices[0].message.content.strip()
    return translated_text

def translate_audio_to_text(audio_data):
    response = openai.Audio.transcribe(
        model="whisper-1",
        file=io.BytesIO(audio_data),
        format="wav"
    )
    return response["text"]

def extract_locations(user_input, input_language="en"):
    if input_language != "en":
        translated_input = translator(user_input)[0]['translation_text']
    else:
        translated_input = user_input

    # Define keywords that indicate a direction or route request
    direction_keywords = ["from", "to", "take", "route", "drive"]

    # Split user input into words
    words = translated_input.lower().split()

    # Initialize variables to store start and end locations
    start_location = None
    end_location = None

    # Use English SpaCy model for extraction
    doc = nlp_en(translated_input)

    for ent in doc.ents:
        if ent.label_ == "LOC":
            if start_location is None:
                start_location = ent.text.capitalize()  # Assuming location is in title case
            elif end_location is None:
                end_location = ent.text.capitalize()  # Assuming location is in title case

    # Fallback to keyword-based extraction for other languages or if no entities found
    if not start_location or not end_location:
        for i in range(len(words)):
            if words[i] in direction_keywords:
                if i + 1 < len(words):
                    if start_location is None:
                        start_location = words[i + 1].capitalize()  # Assuming location is in title case
                    elif end_location is None:
                        end_location = words[i + 1].capitalize()  # Assuming location is in title case

    return start_location, end_location

def generate_route_description(start_location, end_location):
    prompt = f"Provide a detailed driving route from {start_location} to {end_location}. Include major turns, landmarks, and distances."

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ]
    )

    return response.choices[0].message.content.strip()

def main():
    print("Route Description Chatbot. Type 'exit' to quit.")
    while True:
        input_type = input("Enter 'text' for text input or 'audio' for audio input: ").strip().lower()

        if input_type == 'exit':
            print('Exiting')
            break

        if input_type not in ['text', 'audio']:
            print("Invalid input type. Please enter 'text' or 'audio'.")
            continue

       if input_type == 'audio':
            display(HTML(AUDIO_HTML))
            data = eval_js("base64data")
            binary = b64decode(data.split(',')[1])

            process = (
                ffmpeg.input('pipe:0')
                .output('pipe:1', format='wav')
                .run_async(pipe_stdin=True, pipe_stdout=True, pipe_stderr=True, quiet=True, overwrite_output=True, input=binary)
            )
            output, _ = process.communicate()

            riff_chunk_size = len(output) - 8
            q = riff_chunk_size
            b = []
            for i in range(4):
                q, r = divmod(q, 256)
                b.append(r)

            riff = output[:4] + bytes(b) + output[8:]

            sr, audio = wav_read(io.BytesIO(riff))

            audio_data = translate_audio_to_text(audio.tobytes())
            print("Translated Audio:", audio_data)

            # Detect language of audio input
            input_language = "hi" if any('\u0900' <= char <= '\u097f' for char in audio_data) else "en"

            start_location, end_location = extract_locations(audio_data, input_language)
            if not start_location or not end_location:
                print("Could not extract start and end locations from input. Please try again.")
                continue

            route_description = generate_route_description(start_location, end_location)
            print("Route Description:", route_description)

        else:
            user_input = input("Enter your request: ")

            language_code = None
            if ":" in user_input:
                parts = user_input.split(":")
                user_input = parts[0].strip()
                language_code = parts[1].strip().lower()

            start_location, end_location = extract_locations(user_input, language_code)
            if not start_location or not end_location:
                print("Could not extract start and end locations from input. Please try again.")
                continue

            route_description = generate_route_description(start_location, end_location)
            print("Route Description:", route_description)

if __name__ == "__main__":
    main()


Route Description Chatbot. Type 'exit' to quit.
Enter 'text' for text input or 'audio' for audio input: text
Enter your request: text
Could not extract start and end locations from input. Please try again.
Enter 'text' for text input or 'audio' for audio input: text
Enter your request: from kannur to kozhikode
Route Description: I can't provide real-time driving directions, but I can give you a general driving route from one location to another. Please provide me with the specific addresses or locations you want the route for.
Enter 'text' for text input or 'audio' for audio input: text
Enter your request: from kannur to kozhikode
Route Description: I'm unable to provide a detailed driving route from a specific starting point to a specific destination without knowing the locations. Could you please provide me with the addresses or names of the starting point (%1) and the destination (%2) so that I can assist you further?


KeyboardInterrupt: Interrupted by user

In [ ]:


import os
import openai
import re

# Set the OpenAI API key
os.environ["OPENAI_API_KEY"] = ""
openai.api_key = os.getenv("OPENAI_API_KEY")

def extract_locations(user_input):
    # Regular expression to extract locations from input
    pattern = re.compile(r"FROM\s+(.+?)\s+TO\s+(.+)", re.IGNORECASE)
    match = pattern.search(user_input)
    if match:
        start_location = match.group(1).strip()
        end_location = match.group(2).strip()
        return start_location, end_location
    else:
        return None, None

def generate_route_description(start_location, end_location):
    # Prompt for the OpenAI API
    prompt = f"Provide a detailed driving route from {start_location} to {end_location}. Include major turns, landmarks, and distances."

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ]
    )

    return response.choices[0].message.content.strip()

# Run the chatbot in a loop
print("Route Description Chatbot. Type 'exit' to quit.")
while True:
    user_input = input("Enter your request: ")
    if user_input.lower() == 'exit':
        print('Exiting')
        break

    start_location, end_location = extract_locations(user_input)
    if not start_location or not end_location:
        print("Invalid input format. Please use 'HEY TAKE ME FROM [start location] TO [end location]'.")
        continue

    route_description = generate_route_description(start_location, end_location)
    print("Route Description:", route_description)


In [5]:
import os
import openai
import re

# Set the OpenAI API key (replace with your actual key)
os.environ["OPENAI_API_KEY"] = ""
openai.api_key = os.getenv("OPENAI_API_KEY")

def extract_locations(user_input):
    # Regular expressions to extract locations from input
    patterns = [
        re.compile(r"take me from\s+(.+?)\s+to\s+(.+)", re.IGNORECASE),
        re.compile(r"from\s+(.+?)\s+to\s+(.+)", re.IGNORECASE),
        re.compile(r"i want to go from\s+(.+?)\s+to\s+(.+)", re.IGNORECASE)
    ]

    for pattern in patterns:
        match = pattern.search(user_input)
        if match:
            start_location = match.group(1).strip()
            end_location = match.group(2).strip()
            return start_location, end_location
    return None, None

def generate_route_description(start_location, end_location):
    # Prompt for the OpenAI API
    prompt = f"Provide a detailed driving route from {start_location} to {end_location}. Include major turns, landmarks, and distances."

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ]
    )

    return response.choices[0].message.content.strip()

# Run the chatbot in a loop
print("Greetings! I am SARATHI, developed by NIT Calicut students. Type 'exit' to quit.")
while True:
    user_input = input("Enter your request: ")
    if user_input.lower() == 'exit':
        print('Exiting')
        break

    start_location, end_location = extract_locations(user_input)
    if not start_location or not end_location:
        print("Invalid input format. Please use phrases like 'take me from [start location] to [end location]', 'from [start location] to [end location]', or 'I want to go from [start location] to [end location]'.")
        continue

    route_description = generate_route_description(start_location, end_location)
    print("Route Description:", route_description)


Greetings! I am SARATHI, developed by NIT Calicut students. Type 'exit' to quit.
Enter your request: tAKE ME FROM KANNUR TO KOZHIKODE
Route Description: Sure! The approximate driving route from Kannur to Kozhikode is about 90 kilometers and takes roughly 2 to 2.5 hours, depending on traffic conditions. Here is a detailed route:

1. Start your journey in Kannur and head south on the NH 66 (also known as the NH 66 or National Highway 66).

2. After driving for about 25 kilometers, you will reach Thottada. Continue straight on the NH 66.

3. Around the 50-kilometer mark, you will reach Vadakara. Stay on the NH 66.

4. As you continue on the NH 66, after about 75 kilometers, you will reach Beypore. 

5. From Beypore, continue on NH 66 towards Kozhikode. After approximately 90 kilometers, you will reach your destination, Kozhikode.

Here are some major landmarks along the way:
- Kannur
- Thottada
- Vadakara
- Beypore
- Kozhikode

Please note that these are approximate distances and landmark

In [53]:
import os
import openai
import re
from translate import Translator

# Set the OpenAI API key (replace with your actual key)
os.environ["OPENAI_API_KEY"] = ""
openai.api_key = os.getenv("OPENAI_API_KEY")

def extract_locations(user_input):
    # Regular expressions to extract locations from input
    patterns = [
        re.compile(r"take me from\s+(.+?)\s+to\s+(.+)", re.IGNORECASE),
        re.compile(r"from\s+(.+?)\s+to\s+(.+)", re.IGNORECASE),
        re.compile(r"i want to go from\s+(.+?)\s+to\s+(.+)", re.IGNORECASE)
    ]

    for pattern in patterns:
        match = pattern.search(user_input)
        if match:
            start_location = match.group(1).strip()
            end_location = match.group(2).strip()
            return start_location, end_location
    return None, None

def generate_route_description(start_location, end_location):
    # Prompt for the OpenAI API
    prompt = f"Provide a detailed driving route from {start_location} to {end_location}. Include major turns, landmarks, and distances."

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ]
    )

    return response.choices[0].message['content'].strip()

def translate_description(route_description, target_language):
    if target_language == 'en':
        return route_description  # No translation needed if English is selected

    # Translate the route description to the target language
    translator = Translator(to_lang=target_language)
    translated_description = translator.translate(route_description)

    return translated_description

# Run the chatbot in a loop
print("Greetings! I am SARATHI, developed by NIT Calicut students. Type 'exit' to quit.")
while True:
    user_input = input("Enter your request: ")
    if user_input.lower() == 'exit':
        print('Exiting')
        break

    start_location, end_location = extract_locations(user_input)
    if not start_location or not end_location:
        print("Invalid input format. Please use phrases like 'take me from [start location] to [end location]', 'from [start location] to [end location]', or 'I want to go from [start location] to [end location]'.")
        continue

    route_description = generate_route_description(start_location, end_location)
    print("Route Description (in English):", route_description)

    # Ask user for desired language for route description
    target_language = input("Enter language code for translation (e.g., 'hi' for Hindi): ").lower()

    translated_description = translate_description(route_description, target_language)
    print(f"Route Description (in {target_language}):", translated_description)


Greetings! I am SARATHI, developed by NIT Calicut students. Type 'exit' to quit.
Enter your request: from kannur to kozhikode
Route Description (in English): The driving route from Kannur to Kozhikode follows NH66 and is approximately 95 kilometers long. Here is a detailed step-by-step guide with major turns, landmarks, and distances:

1. Start from Kannur and head south on NH66 (also known as the Kozhikode-Kochi Road).
2. Pass by St. Francis Assisi Cathedral on your right.
3. Drive for about 10 kilometers until you reach Thalassery. Look out for Mahe River and Thalassery Sea Bridge.
4. Continue on NH66 and drive through Chala, Kuthuparamba, and Kuttiattoor.
5. After driving approximately 40 kilometers, you will reach Vadakara. Look out for Vadakara Overbridge.
6. Continue on NH66 through regions like Thottilpalam and Chorode.
7. After driving for another 30 kilometers, you will reach Kozhikode. Look out for landmarks like Mavoor Road, Focus Mall, and Calicut Beach.

Please note that r

In [54]:
import os
import openai
import re
from translate import Translator

# Set the OpenAI API key (replace with your actual key)
os.environ["OPENAI_API_KEY"] = ""
openai.api_key = os.getenv("OPENAI_API_KEY")

def extract_locations(user_input):
    # Regular expressions to extract locations from input
    patterns = [
        re.compile(r"take me from\s+(.+?)\s+to\s+(.+)", re.IGNORECASE),
        re.compile(r"from\s+(.+?)\s+to\s+(.+)", re.IGNORECASE),
        re.compile(r"i want to go from\s+(.+?)\s+to\s+(.+)", re.IGNORECASE)
    ]

    for pattern in patterns:
        match = pattern.search(user_input)
        if match:
            start_location = match.group(1).strip()
            end_location = match.group(2).strip()
            return start_location, end_location
    return None, None

def generate_route_description(start_location, end_location):
    # Prompt for the OpenAI API
    prompt = f"Provide a detailed driving route from {start_location} to {end_location}. Include major turns, landmarks, and distances."

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ]
    )

    return response.choices[0].message['content'].strip()

def translate_description(route_description, target_language):
    if target_language == 'en':
        return route_description  # No translation needed if English is selected

    # Initialize translator
    translator = Translator(to_lang=target_language)

    # Split route description into manageable chunks (e.g., sentences)
    sentences = route_description.split('. ')
    translated_sentences = []

    # Translate each sentence and join them back together
    for sentence in sentences:
        translated_sentence = translator.translate(sentence)
        translated_sentences.append(translated_sentence)

    translated_description = '. '.join(translated_sentences)

    return translated_description

# Run the chatbot in a loop
print("Greetings! I am SARATHI, developed by NIT Calicut students. Type 'exit' to quit.")
while True:
    user_input = input("Enter your request: ")
    if user_input.lower() == 'exit':
        print('Exiting')
        break

    start_location, end_location = extract_locations(user_input)
    if not start_location or not end_location:
        print("Invalid input format. Please use phrases like 'take me from [start location] to [end location]', 'from [start location] to [end location]', or 'I want to go from [start location] to [end location]'.")
        continue

    route_description = generate_route_description(start_location, end_location)
    print("Route Description (in English):", route_description)

    # Ask user for desired language for route description
    target_language = input("Enter language code for translation (e.g., 'hi' for Hindi): ").lower()

    translated_description = translate_description(route_description, target_language)
    print(f"Route Description (in {target_language}):", translated_description)


Greetings! I am SARATHI, developed by NIT Calicut students. Type 'exit' to quit.
Enter your request: from kannur to kozhikode
Route Description (in English): Driving from Kannur to Kozhikode is a straightforward journey covering approximately 93 km and usually takes around 2 to 2.5 hours, depending on traffic conditions. Here is a detailed route with major turns, landmarks, and distances:

1. Start from Kannur town center.
2. Head southwest on National Highway 66 (NH 66) towards Kozhikode.
3. After about 16 km, you will pass through Thalassery town.
4. Continue on NH 66 for another 55 km until you reach Vadakara town.
5. In Vadakara, you will pass by Vadakara bus stand and Vadakara railway station.
6. Continue on NH 66 towards Kozhikode for about 22 km.
7. As you approach Kozhikode, you will pass through Ramanattukara and Feroke areas.
8. Follow the signs for Kozhikode city center.
9. In Kozhikode, you will find landmarks like Kozhikode Beach, Mananchira Square, and Kozhikode Town Hall

In [55]:
import os
import openai
import re
from translate import Translator

# Set the OpenAI API key (replace with your actual key)
os.environ["OPENAI_API_KEY"] = ""
openai.api_key = os.getenv("OPENAI_API_KEY")

def extract_locations(user_input):
    # Regular expressions to extract locations from input
    patterns = [
        re.compile(r"take me from\s+(.+?)\s+to\s+(.+)", re.IGNORECASE),
        re.compile(r"from\s+(.+?)\s+to\s+(.+)", re.IGNORECASE),
        re.compile(r"i want to go from\s+(.+?)\s+to\s+(.+)", re.IGNORECASE)
    ]

    for pattern in patterns:
        match = pattern.search(user_input)
        if match:
            start_location = match.group(1).strip()
            end_location = match.group(2).strip()
            return start_location, end_location
    return None, None

def generate_route_description(start_location, end_location):
    # Prompt for the OpenAI API
    prompt = f"Provide a detailed driving route from {start_location} to {end_location}. Include major turns, landmarks, and distances."

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ]
    )

    return response.choices[0].message['content'].strip()

def translate_description(route_description, target_language):
    if target_language == 'en':
        return route_description  # No translation needed if English is selected
    elif target_language == 'ml':
        return translate_to_malayalam(route_description)
    else:
        # Initialize translator
        translator = Translator(to_lang=target_language)

        # Split route description into manageable chunks (e.g., sentences)
        sentences = route_description.split('. ')
        translated_sentences = []

        # Translate each sentence and join them back together
        for sentence in sentences:
            translated_sentence = translator.translate(sentence)
            translated_sentences.append(translated_sentence)

        translated_description = '. '.join(translated_sentences)

        return translated_description

def translate_to_malayalam(route_description):
    # Example of using Google Translate for Malayalam translation
    translator = Translator(to_lang="ml")
    translated_description = translator.translate(route_description)
    return translated_description

# Run the chatbot in a loop
print("Greetings! I am SARATHI, developed by NIT Calicut students. Type 'exit' to quit.")
while True:
    user_input = input("Enter your request: ")
    if user_input.lower() == 'exit':
        print('Exiting')
        break

    start_location, end_location = extract_locations(user_input)
    if not start_location or not end_location:
        print("Invalid input format. Please use phrases like 'take me from [start location] to [end location]', 'from [start location] to [end location]', or 'I want to go from [start location] to [end location]'.")
        continue

    route_description = generate_route_description(start_location, end_location)
    print("Route Description (in English):", route_description)

    # Ask user for desired language for route description
    target_language = input("Enter language code for translation (e.g., 'hi' for Hindi, 'ml' for Malayalam): ").lower()

    translated_description = translate_description(route_description, target_language)
    print(f"Route Description (in {target_language}):", translated_description)


Greetings! I am SARATHI, developed by NIT Calicut students. Type 'exit' to quit.
Enter your request: from thrissur to kochi
Route Description (in English): Sure! Here is a detailed driving route from Thrissur to Kochi:

- Start from Thrissur city center and head south on Shornur Road towards Round West.

- Keep straight on Shornur Road for about 3.5 km until you reach the Kerala Police Academy on your left.

- Continue on Shornur Road for another 2 km until you reach a large roundabout (Chettupuzha Junction). Take the second exit towards Poonkunnam.

- Drive straight on Poonkunnam Road for about 1.5 km until you reach a T-junction. Turn left onto Sakthan Thampuran Road.

- Follow Sakthan Thampuran Road for approximately 1.3 km until you reach a traffic signal near Sakthan Stand. Turn right onto Palace Road.

- Continue on Palace Road for about 700 meters and then turn left onto West Fort Road.

- Drive on West Fort Road for around 1.5 km until you reach a roundabout near Vadakkunnathan

In [56]:
import os
import openai
import re
from translate import Translator

# Set the OpenAI API key (replace with your actual key)
os.environ["OPENAI_API_KEY"] = ""
openai.api_key = os.getenv("OPENAI_API_KEY")

def extract_locations(user_input):
    # Regular expressions to extract locations from input
    patterns = [
        re.compile(r"take me from\s+(.+?)\s+to\s+(.+)", re.IGNORECASE),
        re.compile(r"from\s+(.+?)\s+to\s+(.+)", re.IGNORECASE),
        re.compile(r"i want to go from\s+(.+?)\s+to\s+(.+)", re.IGNORECASE)
    ]

    for pattern in patterns:
        match = pattern.search(user_input)
        if match:
            start_location = match.group(1).strip()
            end_location = match.group(2).strip()
            return start_location, end_location
    return None, None

def generate_route_description(start_location, end_location):
    # Prompt for the OpenAI API
    prompt = f"Provide a detailed driving route from {start_location} to {end_location}. Include major turns, landmarks, and distances."

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ]
    )

    return response.choices[0].message['content'].strip()

def translate_description(route_description, target_language):
    if target_language == 'en':
        return route_description  # No translation needed if English is selected
    elif target_language == 'ml':
        return translate_to_malayalam(route_description)
    else:
        # Initialize translator
        translator = Translator(to_lang=target_language)

        # Split route description into manageable chunks (e.g., sentences)
        sentences = route_description.split('. ')
        translated_sentences = []

        # Translate each sentence and join them back together
        for sentence in sentences:
            translated_sentence = translator.translate(sentence)
            translated_sentences.append(translated_sentence)

        translated_description = '. '.join(translated_sentences)

        return translated_description

def translate_to_malayalam(route_description):
    # Initialize translator for Malayalam
    translator = Translator(to_lang="ml")

    # Split route description into smaller chunks due to query length limitation
    chunk_size = 300  # Adjust chunk size based on the API's limits
    chunks = [route_description[i:i+chunk_size] for i in range(0, len(route_description), chunk_size)]

    translated_chunks = []
    for chunk in chunks:
        translated_chunk = translator.translate(chunk)
        translated_chunks.append(translated_chunk)

    translated_description = '. '.join(translated_chunks)

    return translated_description

# Run the chatbot in a loop
print("Greetings! I am SARATHI, developed by NIT Calicut students. Type 'exit' to quit.")
while True:
    user_input = input("Enter your request: ")
    if user_input.lower() == 'exit':
        print('Exiting')
        break

    start_location, end_location = extract_locations(user_input)
    if not start_location or not end_location:
        print("Invalid input format. Please use phrases like 'take me from [start location] to [end location]', 'from [start location] to [end location]', or 'I want to go from [start location] to [end location]'.")
        continue

    route_description = generate_route_description(start_location, end_location)
    print("Route Description (in English):", route_description)

    # Ask user for desired language for route description
    target_language = input("Enter language code for translation (e.g., 'hi' for Hindi, 'ml' for Malayalam): ").lower()

    translated_description = translate_description(route_description, target_language)
    print(f"Route Description (in {target_language}):", translated_description)


Greetings! I am SARATHI, developed by NIT Calicut students. Type 'exit' to quit.
Enter your request: from thrissur to kochi
Route Description (in English): Here is a detailed driving route from Thrissur to Kochi:

1. Start from Thrissur and head south on SH22 towards Puthenpally Road. (0 km)

2. Continue on SH22 and merge onto NH544 towards Ernakulam/Kochi. (approx. 46 km)

3. Stay on NH544 and pass through Chalakudy, Angamaly, and Aluva. Continue toward Edappally. (approx. 80 km)

4. Take the Edapally - Panvel Highway/SH15 towards Kochi City. (approx. 15 km)

5. Continue on SH15 and take a left onto National Highway 66 (NH66) towards Vyttila Junction. (approx. 6 km)

6. At Vyttila Junction, take the left exit towards Kundannoor Bridge on the Sahodaran Ayyappan Road. (approx. 2 km)

7. Cross the Kundannoor Bridge and continue straight towards Kochi city. (approx. 2 km)

8. You will reach MG Road in Kochi city. Follow the signs to reach your specific destination within Kochi.

Total Dis

In [66]:
import os
import openai
import re
from translate import Translator

os.environ["OPENAI_API_KEY"] = ""
openai.api_key = os.getenv("OPENAI_API_KEY")

def extract_locations(user_input):
    patterns = [
        re.compile(r"take me from\s+(.+?)\s+to\s+(.+)", re.IGNORECASE),
        re.compile(r"from\s+(.+?)\s+to\s+(.+)", re.IGNORECASE),
        re.compile(r"i want to go from\s+(.+?)\s+to\s+(.+)", re.IGNORECASE)
    ]
    for pattern in patterns:
        match = pattern.search(user_input)
        if match:
            start_location = match.group(1).strip()
            end_location = match.group(2).strip()
            return start_location, end_location
    return None, None

def detect_language(user_input):

  if "है" in user_input:  # Simple check for Hindi character (replace with more robust logic)
    return "hi"
  elif "ആണ്" in user_input:  # Simple check for Malayalam character (replace with more robust logic)
    return "ml"
  else:
    return "en"  # Assuming English by default

def translate_text(text, target_language):
  translator = Translator(to_lang=target_language)
  return translator.translate(text)

def generate_route_description(start_location, end_location):
  # Prompt for the OpenAI API (translated to English if needed)
  user_language = detect_language(user_input)
  if user_language != "en":
    translated_prompt = translate_text(f"Provide a detailed driving route from {start_location} to {end_location}. Include major turns, landmarks, and distances.", "en")
    prompt = translated_prompt
  else:
    prompt = f"Provide a detailed driving route from {start_location} to {end_location}. Include major turns, landmarks, and distances."

  response = openai.ChatCompletion.create(
      model="gpt-3.5-turbo",
      messages=[
          {"role": "system", "content": "You are a helpful assistant."},
          {"role": "user", "content": prompt}
      ]
  )

  return response.choices[0].message['content'].strip()


def translate_description(route_description, target_language):
    if target_language == 'en':
        return route_description  # No translation needed if English is selected
    elif target_language == 'ml':
        return translate_to_malayalam(route_description)
    else:
        # Initialize translator
        translator = Translator(to_lang=target_language)

        sentences = route_description.split('. ')
        translated_sentences = []

        # Translate each sentence and join them back together
        for sentence in sentences:
            translated_sentence = translator.translate(sentence)
            translated_sentences.append(translated_sentence)

        translated_description = '. '.join(translated_sentences)

        return translated_description

def translate_to_malayalam(route_description):
    # Initialize translator for Malayalam
    translator = Translator(to_lang="ml")

    # Split route description into smaller chunks due to query length limitation
    chunk_size = 300  # Adjust chunk size based on the API's limits
    chunks = [route_description[i:i+chunk_size] for i in range(0, len(route_description), chunk_size)]

    translated_chunks = []
    for chunk in chunks:
        translated_chunk = translator.translate(chunk)
        translated_chunks.append(translated_chunk)

    translated_description = '. '.join(translated_chunks)

    return translated_description

# Run the chatbot in a loop
print("Greetings! I am SARATHI, developed by NIT Calicut students. Type 'exit' to quit.")
while True:
    user_input = input("Enter your request: ")
    if user_input.lower() == 'exit':
        print('Exiting')
        break

    start_location, end_location = extract_locations(user_input)
    if not start_location or not end_location:
        print("Invalid input format. Please use phrases like 'take me from [start location] to [end location]', 'from [start location] to [end location]', or 'I want to go from [start location] to [end location]'.")
        continue

    route_description = generate_route_description(start_location, end_location)
    print("Route Description (in English):", route_description)

    # Ask user for desired language for route description
    target_language = input("Enter language code for translation (e.g., 'hi' for Hindi, 'ml' for Malayalam): ").lower()

    translated_description = translate_description(route_description, target_language)
    print(f"Route Description (in {target_language}):", translated_description)


IndentationError: unexpected indent (<ipython-input-66-2476b9a26499>, line 17)

In [71]:
import os
import openai
import re
from translate import Translator

# Set the OpenAI API key (replace with your actual key)
os.environ["OPENAI_API_KEY"] = ""
openai.api_key = os.getenv("OPENAI_API_KEY")

def detect_language(text):
    # This function uses OpenAI's model to detect the language of the input text
    response = openai.Completion.create(
        engine="gpt-3.5-turbo-instruct",
        prompt=f"Detect the language of the following text:\n\n{text}",
        max_tokens=5
    )
    return response.choices[0].text.strip()

def translate_to_english(text, source_language):
    translator = Translator(from_lang=source_language, to_lang="en")
    return translator.translate(text)

def extract_locations(user_input):
    # Regular expressions to extract locations from input
    patterns = [
        re.compile(r"take me from\s+(.+?)\s+to\s+(.+)", re.IGNORECASE),
        re.compile(r"from\s+(.+?)\s+to\s+(.+)", re.IGNORECASE),
        re.compile(r"i want to go from\s+(.+?)\s+to\s+(.+)", re.IGNORECASE)
    ]

    for pattern in patterns:
        match = pattern.search(user_input)
        if match:
            start_location = match.group(1).strip()
            end_location = match.group(2).strip()
            return start_location, end_location
    return None, None

def generate_route_description(start_location, end_location):
    # Prompt for the OpenAI API
    prompt = f"Provide a detailed driving route from {start_location} to {end_location}. Include major turns, landmarks, and distances."

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ]
    )

    return response.choices[0].message['content'].strip()

def translate_description(route_description, target_language):
    if target_language == 'en':
        return route_description  # No translation needed if English is selected
    elif target_language == 'ml':
        return translate_to_malayalam(route_description)
    else:
        # Initialize translator
        translator = Translator(to_lang=target_language)

        # Split route description into manageable chunks (e.g., sentences)
        sentences = route_description.split('. ')
        translated_sentences = []

        # Translate each sentence and join them back together
        for sentence in sentences:
            translated_sentence = translator.translate(sentence)
            translated_sentences.append(translated_sentence)

        translated_description = '. '.join(translated_sentences)

        return translated_description

def translate_to_malayalam(route_description):
    # Initialize translator for Malayalam
    translator = Translator(to_lang="ml")

    # Split route description into smaller chunks due to query length limitation
    chunk_size = 300  # Adjust chunk size based on the API's limits
    chunks = [route_description[i:i+chunk_size] for i in range(0, len(route_description), chunk_size)]

    translated_chunks = []
    for chunk in chunks:
        translated_chunk = translator.translate(chunk)
        translated_chunks.append(translated_chunk)

    translated_description = '. '.join(translated_chunks)

    return translated_description

# Run the chatbot in a loop
print("Greetings! I am SARATHI, developed by NIT Calicut students. Type 'exit' to quit.")
while True:
    user_input = input("Enter your request: ")
    if user_input.lower() == 'exit':
        print('Exiting')
        break

    detected_language = detect_language(user_input)
    if detected_language in ['Hindi', 'Malayalam']:
        user_input = translate_to_english(user_input, detected_language.lower())

    start_location, end_location = extract_locations(user_input)
    if not start_location or not end_location:
        print("Invalid input format. Please use phrases like 'take me from [start location] to [end location]', 'from [start location] to [end location]', or 'I want to go from [start location] to [end location]'.")
        continue

    route_description = generate_route_description(start_location, end_location)
    print("Route Description (in English):", route_description)

    # Ask user for desired language for route description
    target_language = input("Enter language code for translation (e.g., 'hi' for Hindi, 'ml' for Malayalam): ").lower()

    translated_description = translate_description(route_description, target_language)
    print(f"Route Description (in {target_language}):", translated_description)


Greetings! I am SARATHI, developed by NIT Calicut students. Type 'exit' to quit.
Enter your request: തൃശൂർ മുതൽ കൊച്ചി വരെ


RuntimeError: generator raised StopIteration

In [74]:
import os
import openai
import re
from translate import Translator

# Set the OpenAI API key (replace with your actual key)
os.environ["OPENAI_API_KEY"] = " "
openai.api_key = os.getenv("OPENAI_API_KEY")

def detect_language(text):
    response = openai.Completion.create(
        engine="gpt-3.5-turbo-instruct",
        prompt=f"Detect the language of the following text:\n\n{text}",
        max_tokens=5
    )
    return response.choices[0].text.strip()

def translate_to_english(text, source_language):
    translator = Translator(from_lang=source_language, to_lang="en")
    try:
        return translator.translate(text)
    except Exception as e:
        print(f"Error translating to English: {e}")
        return text

def extract_locations(user_input):
    patterns = [
        re.compile(r"take me from\s+(.+?)\s+to\s+(.+)", re.IGNORECASE),
        re.compile(r"from\s+(.+?)\s+to\s+(.+)", re.IGNORECASE),
        re.compile(r"i want to go from\s+(.+?)\s+to\s+(.+)", re.IGNORECASE)
    ]

    for pattern in patterns:
        match = pattern.search(user_input)
        if match:
            start_location = match.group(1).strip()
            end_location = match.group(2).strip()
            return start_location, end_location
    return None, None

def generate_route_description(start_location, end_location):
    prompt = f"Provide a detailed driving route from {start_location} to {end_location}. Include major turns, landmarks, and distances."

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ]
    )

    return response.choices[0].message['content'].strip()

def translate_description(route_description, target_language):
    if target_language == 'en':
        return route_description
    elif target_language == 'ml':
        return translate_to_malayalam(route_description)
    else:
        translator = Translator(to_lang=target_language)

        sentences = route_description.split('. ')
        translated_sentences = []

        for sentence in sentences:
            try:
                translated_sentence = translator.translate(sentence)
                translated_sentences.append(translated_sentence)
            except Exception as e:
                print(f"Error translating sentence: {e}")
                translated_sentences.append(sentence)

        translated_description = '. '.join(translated_sentences)

        return translated_description

def translate_to_malayalam(route_description):
    translator = Translator(to_lang="ml")

    chunk_size = 300
    chunks = [route_description[i:i+chunk_size] for i in range(0, len(route_description), chunk_size)]

    translated_chunks = []
    for chunk in chunks:
        try:
            translated_chunk = translator.translate(chunk)
            translated_chunks.append(translated_chunk)
        except Exception as e:
            print(f"Error translating chunk: {e}")
            translated_chunks.append(chunk)

    translated_description = '. '.join(translated_chunks)

    return translated_description

print("Greetings! I am SARATHI, developed by NIT Calicut students. Type 'exit' to quit.")
while True:
    user_input = input("Enter your request: ")
    if user_input.lower() == 'exit':
        print('Exiting')
        break

    detected_language = detect_language(user_input)
    if detected_language.lower() in ['hindi', 'malayalam']:
        user_input = translate_to_english(user_input, detected_language.lower())

    start_location, end_location = extract_locations(user_input)
    if not start_location or not end_location:
        print("Invalid input format. Please use phrases like 'take me from [start location] to [end location]', 'from [start location] to [end location]', or 'I want to go from [start location] to [end location]'.")
        continue

    route_description = generate_route_description(start_location, end_location)
    print("Route Description (in English):", route_description)

    target_language = input("Enter language code for translation (e.g., 'hi' for Hindi, 'ml' for Malayalam): ").lower()

    translated_description = translate_description(route_description, target_language)
    print(f"Route Description (in {target_language}):", translated_description)


Greetings! I am SARATHI, developed by NIT Calicut students. Type 'exit' to quit.
Enter your request: തൃശൂർ മുതൽ കൊച്ചി വരെ
Error translating to English: generator raised StopIteration
Invalid input format. Please use phrases like 'take me from [start location] to [end location]', 'from [start location] to [end location]', or 'I want to go from [start location] to [end location]'.
Enter your request: തൃശൂർ മുതൽ കൊച്ചി വരെ
Error translating to English: generator raised StopIteration
Invalid input format. Please use phrases like 'take me from [start location] to [end location]', 'from [start location] to [end location]', or 'I want to go from [start location] to [end location]'.
Enter your request: exit
Exiting


In [75]:
!pip install googletrans==4.0.0-rc1

  Preparing metadata (setup.py) ... done
  Created wheel for googletrans: filename=googletrans-4.0.0rc1-py3-none-any.whl size=17395 sha256=a7f754f84eb1c3dc42a53c49d9dce229a8279d9b0c252b951bbc2fde1e71ebd5
  Stored in directory: /root/.cache/pip/wheels/c0/59/9f/7372f0cf70160fe61b528532e1a7c8498c4becd6bcffb022de
Successfully built googletrans
  Attempting uninstall: googletrans
    Found existing installation: googletrans 3.0.0
    Uninstalling googletrans-3.0.0:
      Successfully uninstalled googletrans-3.0.0


In [77]:
import os
import openai
import re
from googletrans import Translator

# Set the OpenAI API key (replace with your actual key)
os.environ["OPENAI_API_KEY"] = ""
openai.api_key = os.getenv("OPENAI_API_KEY")

translator = Translator()

def detect_language(text):
    response = openai.Completion.create(
        engine="gpt-3.5-turbo-instruct",
        prompt=f"Detect the language of the following text:\n\n{text}",
        max_tokens=5
    )
    return response.choices[0].text.strip()

def translate_to_english(text, source_language):
    try:
        translation = translator.translate(text, src=source_language, dest='en')
        return translation.text
    except Exception as e:
        print(f"Error translating to English: {e}")
        return text

def extract_locations(user_input):
    patterns = [
        re.compile(r"take me from\s+(.+?)\s+to\s+(.+)", re.IGNORECASE),
        re.compile(r"from\s+(.+?)\s+to\s+(.+)", re.IGNORECASE),
        re.compile(r"i want to go from\s+(.+?)\s+to\s+(.+)", re.IGNORECASE)
    ]

    for pattern in patterns:
        match = pattern.search(user_input)
        if match:
            start_location = match.group(1).strip()
            end_location = match.group(2).strip()
            return start_location, end_location
    return None, None

def generate_route_description(start_location, end_location):
    prompt = f"Provide a detailed driving route from {start_location} to {end_location}. Include major turns, landmarks, and distances."

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ]
    )

    return response.choices[0].message['content'].strip()

def translate_description(route_description, target_language):
    if target_language == 'en':
        return route_description
    else:
        try:
            translation = translator.translate(route_description, dest=target_language)
            return translation.text
        except Exception as e:
            print(f"Error translating description: {e}")
            return route_description

print("Greetings! I am SARATHI, developed by NIT Calicut students. Type 'exit' to quit.")
while True:
    user_input = input("Enter your request: ")
    if user_input.lower() == 'exit':
        print('Exiting')
        break

    detected_language = detect_language(user_input)
    if detected_language.lower() in ['hindi', 'malayalam']:
        user_input = translate_to_english(user_input, detected_language.lower())

    start_location, end_location = extract_locations(user_input)
    if not start_location or not end_location:
        print("Invalid input format. Please use phrases like 'take me from [start location] to [end location]', 'from [start location] to [end location]', or 'I want to go from [start location] to [end location]'.")
        continue

    route_description = generate_route_description(start_location, end_location)
    print("Route Description (in English):", route_description)

    target_language = input("Enter language code for translation (e.g., 'hi' for Hindi, 'ml' for Malayalam): ").lower()

    translated_description = translate_description(route_description, target_language)
    print(f"Route Description (in {target_language}):", translated_description)


Greetings! I am SARATHI, developed by NIT Calicut students. Type 'exit' to quit.
Enter your request: തൃശൂർ മുതൽ കൊച്ചി വരെ
Invalid input format. Please use phrases like 'take me from [start location] to [end location]', 'from [start location] to [end location]', or 'I want to go from [start location] to [end location]'.
Enter your request: exit
Exiting


In [64]:
import os
import openai
import re
from translate import Translator
from langdetect import detect  # For language detection

# Set the OpenAI API key (replace with your actual key)
os.environ["OPENAI_API_KEY"] = ""
openai.api_key = os.getenv("OPENAI_API_KEY")

def extract_locations(user_input):
    # Regular expressions to extract locations from input
    patterns = [
        re.compile(r"take me from\s+(.+?)\s+to\s+(.+)", re.IGNORECASE),
        re.compile(r"from\s+(.+?)\s+to\s+(.+)", re.IGNORECASE),
        re.compile(r"i want to go from\s+(.+?)\s+to\s+(.+)", re.IGNORECASE)
    ]

    for pattern in patterns:
        match = pattern.search(user_input)
        if match:
            start_location = match.group(1).strip()
            end_location = match.group(2).strip()
            return start_location, end_location
    return None, None

def translate_to_english(user_input, source_language):
    # Initialize translator for source language
    translator = Translator(from_lang=source_language, to_lang="en")
    translated_input = translator.translate(user_input)
    return translated_input

def generate_route_description(start_location, end_location):
    # Prompt for the OpenAI API
    prompt = f"Provide a detailed driving route from {start_location} to {end_location}. Include major turns, landmarks, and distances."

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ]
    )

    return response.choices[0].message['content'].strip()

def translate_description(route_description, target_language):
    if target_language == 'en':
        return route_description  # No translation needed if English is selected
    elif target_language == 'ml':
        return translate_to_malayalam(route_description)
    elif target_language == 'hi':
        return translate_to_hindi(route_description)
    else:
        # Initialize translator
        translator = Translator(to_lang=target_language)

        # Split route description into manageable chunks (e.g., sentences)
        sentences = route_description.split('. ')
        translated_sentences = []

        # Translate each sentence and join them back together
        for sentence in sentences:
            translated_sentence = translator.translate(sentence)
            translated_sentences.append(translated_sentence)

        translated_description = '. '.join(translated_sentences)

        return translated_description

def translate_to_malayalam(route_description):
    # Initialize translator for Malayalam
    translator = Translator(to_lang="ml")

    # Split route description into smaller chunks due to query length limitation
    chunk_size = 300  # Adjust chunk size based on the API's limits
    chunks = [route_description[i:i+chunk_size] for i in range(0, len(route_description), chunk_size)]

    translated_chunks = []
    for chunk in chunks:
        translated_chunk = translator.translate(chunk)
        translated_chunks.append(translated_chunk)

    translated_description = '. '.join(translated_chunks)

    return translated_description

def translate_to_hindi(route_description):
    # Initialize translator for Hindi
    translator = Translator(to_lang="hi")

    # Split route description into smaller chunks due to query length limitation
    chunk_size = 300  # Adjust chunk size based on the API's limits
    chunks = [route_description[i:i+chunk_size] for i in range(0, len(route_description), chunk_size)]

    translated_chunks = []
    for chunk in chunks:
        translated_chunk = translator.translate(chunk)
        translated_chunks.append(translated_chunk)

    translated_description = '. '.join(translated_chunks)

    return translated_description

# Run the chatbot in a loop
print("Greetings! I am SARATHI, developed by NIT Calicut students. Type 'exit' to quit.")
while True:
    user_input = input("Enter your request in Malayalam (മലയാളം) or Hindi (हिन्दी): ")

    if user_input.lower() == 'exit':
        print('Exiting')
        break

    # Detect the language of user input
    try:
        user_language = detect(user_input)
    except:
        print("Language detection failed. Please enter your request in Malayalam (മലയാളം) or Hindi (हिन्दी).")
        continue

    # Translate the user input to English for processing
    translated_input = translate_to_english(user_input, user_language)

    start_location, end_location = extract_locations(translated_input)

    if not start_location or not end_location:
        print("Invalid input format. Please use phrases like 'take me from [start location] to [end location]', 'from [start location] to [end location]', or 'I want to go from [start location] to [end location]'.")
        continue

    route_description = generate_route_description(start_location, end_location)
    print("Route Description (in English):", route_description)

    # Ask user for desired language for route description
    target_language = input("Enter language code for translation (e.g., 'hi' for Hindi, 'ml' for Malayalam): ").lower()

    translated_description = translate_description(route_description, target_language)
    print(f"Route Description (in {target_language}):", translated_description)


Greetings! I am SARATHI, developed by NIT Calicut students. Type 'exit' to quit.
Enter your request in Malayalam (മലയാളം) or Hindi (हिन्दी): തൃശൂർ മുതൽ കൊച്ചി വരെ
Invalid input format. Please use phrases like 'take me from [start location] to [end location]', 'from [start location] to [end location]', or 'I want to go from [start location] to [end location]'.


KeyboardInterrupt: Interrupted by user

In [59]:
import os
import openai
import re
from translate import Translator
from langdetect import detect


os.environ["OPENAI_API_KEY"] =
openai.api_key = os.getenv("OPENAI_API_KEY")

def extract_locations(user_input):

    patterns = [
        re.compile(r"take me from\s+(.+?)\s+to\s+(.+)", re.IGNORECASE),
        re.compile(r"from\s+(.+?)\s+to\s+(.+)", re.IGNORECASE),
        re.compile(r"i want to go from\s+(.+?)\s+to\s+(.+)", re.IGNORECASE)
    ]

    for pattern in patterns:
        match = pattern.search(user_input)
        if match:
            start_location = match.group(1).strip()
            end_location = match.group(2).strip()
            return start_location, end_location
    return None, None

def generate_route_description(start_location, end_location):

    prompt = f"Provide a detailed driving route from {start_location} to {end_location}. Include major turns, landmarks, and distances."

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ]
    )

    return response.choices[0].message['content'].strip()

def translate_description(route_description, target_language):
    if target_language == 'en':
        return route_description
    elif target_language == 'ml':
        return translate_to_malayalam(route_description)
    elif target_language == 'hi':
        return translate_to_hindi(route_description)
    else:

        translator = Translator(to_lang=target_language)

        sentences = route_description.split('. ')
        translated_sentences = []


        for sentence in sentences:
            translated_sentence = translator.translate(sentence)
            translated_sentences.append(translated_sentence)

        translated_description = '. '.join(translated_sentences)

        return translated_description

def translate_to_malayalam(route_description):

    translator = Translator(to_lang="ml")


    chunk_size = 300
    chunks = [route_description[i:i+chunk_size] for i in range(0, len(route_description), chunk_size)]

    translated_chunks = []
    for chunk in chunks:
        translated_chunk = translator.translate(chunk)
        translated_chunks.append(translated_chunk)

    translated_description = '. '.join(translated_chunks)

    return translated_description

def translate_to_hindi(route_description):

    translator = Translator(to_lang="hi")


    chunk_size = 300
    chunks = [route_description[i:i+chunk_size] for i in range(0, len(route_description), chunk_size)]

    translated_chunks = []
    for chunk in chunks:
        translated_chunk = translator.translate(chunk)
        translated_chunks.append(translated_chunk)

    translated_description = '. '.join(translated_chunks)

    return translated_description

print("Greetings! I am SARATHI, developed by NIT Calicut students. Type 'exit' to quit.")
while True:
    user_input = input("Enter your request: ")

    if user_input.lower() == 'exit':
        print('Exiting')
        break


    try:
        user_language = detect(user_input)
    except:
        print("Language detection failed. Please enter your request in Malayalam (മലയാളം) or Hindi (हिन्दी).")
        continue

    if user_language == 'ml':
        user_input = translate_to_english(user_input, 'ml')
    elif user_language == 'hi':
        user_input = translate_to_english(user_input, 'hi')

    start_location, end_location = extract_locations(user_input)

    if not start_location or not end_location:
        print("Invalid input format. Please use phrases like 'take me from [start location] to [end location]', 'from [start location] to [end location]', or 'I want to go from [start location] to [end location]'.")
        continue

    route_description = generate_route_description(start_location, end_location)
    print("Route Description (in English):", route_description)

    target_language = input("Enter language code for translation (e.g., 'hi' for Hindi, 'ml' for Malayalam): ").lower()

    translated_description = translate_description(route_description, target_language)
    print(f"Route Description (in {target_language}):", translated_description)


Greetings! I am SARATHI, developed by NIT Calicut students. Type 'exit' to quit.
Enter your request: തൃശൂർ മുതൽ കൊച്ചി വരെ


NameError: name 'translate_to_english' is not defined

In [60]:
import os
import openai
import re
from translate import Translator
from langdetect import detect  # For language detection

# Set the OpenAI API key (replace with your actual key)
os.environ["OPENAI_API_KEY"] = ""
openai.api_key = os.getenv("OPENAI_API_KEY")

def extract_locations(user_input):
    # Regular expressions to extract locations from input
    patterns = [
        re.compile(r"take me from\s+(.+?)\s+to\s+(.+)", re.IGNORECASE),
        re.compile(r"from\s+(.+?)\s+to\s+(.+)", re.IGNORECASE),
        re.compile(r"i want to go from\s+(.+?)\s+to\s+(.+)", re.IGNORECASE)
    ]

    for pattern in patterns:
        match = pattern.search(user_input)
        if match:
            start_location = match.group(1).strip()
            end_location = match.group(2).strip()
            return start_location, end_location
    return None, None

def generate_route_description(start_location, end_location):
    # Prompt for the OpenAI API
    prompt = f"Provide a detailed driving route from {start_location} to {end_location}. Include major turns, landmarks, and distances."

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ]
    )

    return response.choices[0].message['content'].strip()

def translate_description(route_description, target_language):
    if target_language == 'en':
        return route_description  # No translation needed if English is selected
    elif target_language == 'ml':
        return translate_to_malayalam(route_description)
    elif target_language == 'hi':
        return translate_to_hindi(route_description)
    else:
        # Initialize translator
        translator = Translator(to_lang=target_language)

        # Split route description into manageable chunks (e.g., sentences)
        sentences = route_description.split('. ')
        translated_sentences = []

        # Translate each sentence and join them back together
        for sentence in sentences:
            translated_sentence = translator.translate(sentence)
            translated_sentences.append(translated_sentence)

        translated_description = '. '.join(translated_sentences)

        return translated_description

def translate_to_malayalam(route_description):
    # Initialize translator for Malayalam
    translator = Translator(to_lang="ml")

    # Split route description into smaller chunks due to query length limitation
    chunk_size = 300  # Adjust chunk size based on the API's limits
    chunks = [route_description[i:i+chunk_size] for i in range(0, len(route_description), chunk_size)]

    translated_chunks = []
    for chunk in chunks:
        translated_chunk = translator.translate(chunk)
        translated_chunks.append(translated_chunk)

    translated_description = '. '.join(translated_chunks)

    return translated_description

def translate_to_hindi(route_description):
    # Initialize translator for Hindi
    translator = Translator(to_lang="hi")

    # Split route description into smaller chunks due to query length limitation
    chunk_size = 300  # Adjust chunk size based on the API's limits
    chunks = [route_description[i:i+chunk_size] for i in range(0, len(route_description), chunk_size)]

    translated_chunks = []
    for chunk in chunks:
        translated_chunk = translator.translate(chunk)
        translated_chunks.append(translated_chunk)

    translated_description = '. '.join(translated_chunks)

    return translated_description

# Run the chatbot in a loop
print("Greetings! I am SARATHI, developed by NIT Calicut students. Type 'exit' to quit.")
while True:
    user_input = input("Enter your request: ")

    if user_input.lower() == 'exit':
        print('Exiting')
        break

    # Detect the language of user input
    try:
        user_language = detect(user_input)
    except:
        print("Language detection failed. Please enter your request in Malayalam (മലയാളം) or Hindi (हिन्दी).")
        continue

    start_location, end_location = extract_locations(user_input)

    if not start_location or not end_location:
        print("Invalid input format. Please use phrases like 'take me from [start location] to [end location]', 'from [start location] to [end location]', or 'I want to go from [start location] to [end location]'.")
        continue

    route_description = generate_route_description(start_location, end_location)
    print("Route Description (in English):", route_description)

    # Ask user for desired language for route description
    target_language = input("Enter language code for translation (e.g., 'hi' for Hindi, 'ml' for Malayalam): ").lower()

    translated_description = translate_description(route_description, target_language)
    print(f"Route Description (in {target_language}):", translated_description)


Greetings! I am SARATHI, developed by NIT Calicut students. Type 'exit' to quit.
Enter your request: തൃശൂർ മുതൽ കൊച്ചി വരെ
Invalid input format. Please use phrases like 'take me from [start location] to [end location]', 'from [start location] to [end location]', or 'I want to go from [start location] to [end location]'.
Enter your request: from thrissur to kochi
Route Description (in English): Here is a detailed driving route from Thrissur to Kochi:

1. Start in Thrissur and head south on SH 22 (Shoranur - Thrissur Highway) towards Muttikkal - 0 km
2. Continue on the same road for about 10 km until you reach Palakkad Road - 10 km
3. Take a left onto Palakkad Road and continue for about 9 km
4. At the roundabout, take the 2nd exit onto NH 544 (Thrissur - Kochi Highway) - 19 km
5. Continue on NH 544 for about 40 km until you reach Angamaly - 59 km
6. At Angamaly, take the NH 966A towards Kochi - 60 km
7. Continue on NH 966A for about 18 km until you reach the outskirts of Kochi - 78 km
8

In [58]:
!pip install langdetect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 11.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993227 sha256=52de7c424b09b00e2d2fb97f001a37ed7e327c578d7a16c3b8b3d84172d75a25
  Stored in directory: /root/.cache/pip/wheels/95/03/7d/59ea870c70ce4e5a370638b5462a7711ab78fba2f655d05106
Successfully built langdetect


In [32]:
!pip install transformers torch

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21

In [33]:
import os
import re
import torch
from transformers import MarianMTModel, MarianTokenizer

# Set up the MarianMT model and tokenizer
model_name = 'Helsinki-NLP/opus-mt-en-es'  # English to Spanish translation
model = MarianMTModel.from_pretrained(model_name)
tokenizer = MarianTokenizer.from_pretrained(model_name)

def extract_locations(user_input):
    # Regular expressions to extract locations from input
    patterns = [
        re.compile(r"take me from\s+(.+?)\s+to\s+(.+)", re.IGNORECASE),
        re.compile(r"from\s+(.+?)\s+to\s+(.+)", re.IGNORECASE),
        re.compile(r"i want to go from\s+(.+?)\s+to\s+(.+)", re.IGNORECASE)
    ]

    for pattern in patterns:
        match = pattern.search(user_input)
        if match:
            start_location = match.group(1).strip()
            end_location = match.group(2).strip()
            return start_location, end_location
    return None, None

def generate_route_description(start_location, end_location, target_language='es'):
    # Prompt for the model
    prompt = f"Provide a detailed driving route from {start_location} to {end_location}. Include major turns, landmarks, and distances."

    # Tokenize the prompt
    inputs = tokenizer.prepare_seq2seq_batch(src_texts=[prompt], return_tensors="pt")

    # Generate response
    with torch.no_grad():
        outputs = model.generate(**inputs)

    # Decode and convert response tensor to text
    translated_description = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]

    return translated_description

# Run the chatbot in a loop
print("Greetings! I am SARATHI, developed by NIT Calicut students. Type 'exit' to quit.")
while True:
    user_input = input("Enter your request: ")
    if user_input.lower() == 'exit':
        print('Exiting')
        break

    start_location, end_location = extract_locations(user_input)
    if not start_location or not end_location:
        print("Invalid input format. Please use phrases like 'take me from [start location] to [end location]', 'from [start location] to [end location]', or 'I want to go from [start location] to [end location]'.")
        continue

    # Generate route description in Spanish
    route_description = generate_route_description(start_location, end_location, target_language='es')
    print("Route Description (in Spanish):", route_description)


config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/312M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/802k [00:00<?, ?B/s]

...

vocab.json:   0%|          | 0.00/1.59M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


Greetings! I am SARATHI, developed by NIT Calicut students. Type 'exit' to quit.
Enter your request: take me from kannur to kozhikode


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:4072: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the regular
`__call__` method to prepare your inputs and targets.

Here is a short example:

model_inputs = tokenizer(src_texts, text_target=tgt_texts, ...)

If you either need to use different keyword arguments for the source and target texts, you should do two calls like
this:

model_inputs = tokenizer(src_texts, ...)
labels = tokenizer(text_target=tgt_texts, ...)
model_inputs["labels"] = labels["input_ids"]

See the documentation of your specific tokenizer for more details on the specific arguments to the tokenizer of choice.
For a more complete example, see the implementation of `prepare_seq2seq_batch`.

  warnings.warn(formatted_warning, FutureWarning)


Route Description (in Spanish): Proveer una ruta de conducción detallada de Kannur a kozhikode. Incluye grandes giros, puntos de referencia y distancias.
Enter your request: exit
Exiting


In [35]:
exit


Greetings! I am SARATHI, developed by NIT Calicut students. Type 'exit' to quit.
Enter your request: take me from kannur to kozhikode
Route Description (in Spanish): QUERY LENGTH LIMIT EXCEEDED. MAX ALLOWED QUERY : 500 CHARS अपनी यात्रा के दौरान रीयल - टाइम अपडेट और दिशा - निर्देश पाने के लिए नेविगेशन ऐप का इस्तेमाल करने की सलाह दी जाती है।
Enter your request: exit
Exiting


In [36]:
import os
import re
import openai
from translate import Translator

# Set the OpenAI API key (replace with your actual key)
os.environ["OPENAI_API_KEY"] = ""
openai.api_key = os.getenv("OPENAI_API_KEY")

def extract_locations(user_input):
    # Regular expressions to extract locations from input
    patterns = [
        re.compile(r"take me from\s+(.+?)\s+to\s+(.+)", re.IGNORECASE),
        re.compile(r"from\s+(.+?)\s+to\s+(.+)", re.IGNORECASE),
        re.compile(r"i want to go from\s+(.+?)\s+to\s+(.+)", re.IGNORECASE)
    ]

    for pattern in patterns:
        match = pattern.search(user_input)
        if match:
            start_location = match.group(1).strip()
            end_location = match.group(2).strip()
            return start_location, end_location
    return None, None

def ask_language():
    while True:
        print("In which language would you like the route description?")
        print("1. English")
        print("2. Spanish")
        print("3. Hindi")
        choice = input("Enter the number corresponding to your choice (1/2/3): ").strip()

        if choice == '1':
            return 'en'
        elif choice == '2':
            return 'es'
        elif choice == '3':
            return 'hi'
        else:
            print("Invalid choice. Please enter 1, 2, or 3.")

def generate_route_description(start_location, end_location, target_language='en'):
    # Prompt for the OpenAI API
    prompt = f"Provide a detailed driving route from {start_location} to {end_location}. Include major turns, landmarks, and distances."

    # Generate response from OpenAI
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ]
    )
    route_description = response.choices[0].message.content.strip()

    # Translate the route description to the target language if needed
    if target_language != 'en':
        translator = Translator(to_lang=target_language)
        translated_description = translator.translate(route_description)
    else:
        translated_description = route_description

    return translated_description

# Run the chatbot in a loop
print("Greetings! I am SARATHI, developed by NIT Calicut students. Type 'exit' to quit.")
while True:
    user_input = input("Enter your request: ")
    if user_input.lower() == 'exit':
        print('Exiting')
        break

    start_location, end_location = extract_locations(user_input)
    if not start_location or not end_location:
        print("Invalid input format. Please use phrases like 'take me from [start location] to [end location]', 'from [start location] to [end location]', or 'I want to go from [start location] to [end location]'.")
        continue

    target_language = ask_language()
    route_description = generate_route_description(start_location, end_location, target_language)

    language_name = {
        'en': 'English',
        'es': 'Spanish',
        'hi': 'Hindi'
    }.get(target_language, 'Unknown')

    print(f"Route Description (in {language_name}):", route_description)


Greetings! I am SARATHI, developed by NIT Calicut students. Type 'exit' to quit.
Enter your request: from kannur to kozhikode
In which language would you like the route description?
1. English
2. Spanish
3. Hindi
Enter the number corresponding to your choice (1/2/3): 1
Route Description (in English): The driving route from Kannur to Kozhikode is approximately 90 kilometers long, and here is a detailed guide with major turns, landmarks, and distances:

1. Start from Kannur and head south on NH 66 (also known as Kochi - Panvel Highway).
2. After approximately 10 kilometers, you will pass by Thottada Beach on your left.
3. Continue on NH 66 for another 15 kilometers until you reach Thalassery.
4. Pass through Thalassery town and continue on NH 66.
5. After around 35 kilometers from Kannur, you will reach Mahe town.
6. Continue on NH 66 for another 15 kilometers, passing by Vadakara town.
7. After around 15 kilometers from Vadakara, you will reach Kozhikode city.
8. Follow the signs toward

In [39]:
import os
import re
import openai
from translate import Translator

# Set the OpenAI API key (replace with your actual key)
os.environ["OPENAI_API_KEY""
openai.api_key = os.getenv("OPENAI_API_KEY")

def extract_locations(user_input):
    # Regular expressions to extract locations from input
    patterns = [
        re.compile(r"take me from\s+(.+?)\s+to\s+(.+)", re.IGNORECASE),
        re.compile(r"from\s+(.+?)\s+to\s+(.+)", re.IGNORECASE),
        re.compile(r"i want to go from\s+(.+?)\s+to\s+(.+)", re.IGNORECASE)
    ]

    for pattern in patterns:
        match = pattern.search(user_input)
        if match:
            start_location = match.group(1).strip()
            end_location = match.group(2).strip()
            return start_location, end_location
    return None, None

def generate_route_description(start_location, end_location, target_language='en'):
    # Prompt for the OpenAI API
    prompt = f"Provide a detailed driving route from {start_location} to {end_location}. Include major turns, landmarks, and distances."

    # Generate response from OpenAI using the chat completion endpoint
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ]
    )
    route_description = response.choices[0].message['content'].strip()

    # Translate the route description to the target language if needed
    if target_language != 'en':
        translator = Translator(to_lang=target_language)
        translated_description = translator.translate(route_description)
    else:
        translated_description = route_description

    return translated_description

# Run the chatbot in a loop
print("Greetings! I am SARATHI, developed by NIT Calicut students. Type 'exit' to quit.")
while True:
    user_input = input("Enter your request: ")
    if user_input.lower() == 'exit':
        print('Exiting')
        break

    start_location, end_location = extract_locations(user_input)
    if not start_location or not end_location:
        print("Invalid input format. Please use phrases like 'take me from [start location] to [end location]', 'from [start location] to [end location]', or 'I want to go from [start location] to [end location]'.")
        continue

    # Example: Translating to Malayalam ('ml'). You can change 'ml' to any other language code as needed.
    route_description = generate_route_description(start_location, end_location, target_language='ml')

    language_name = {
        'en': 'English',
        'ml': 'Malayalam'
    }.get('ml', 'Unknown')

    print(f"Route Description (in {language_name}): {route_description}")


Greetings! I am SARATHI, developed by NIT Calicut students. Type 'exit' to quit.
Enter your request: from thrissur to kochi
Route Description (in Malayalam): QUERY LENGTH LIMIT EXCEEDED. MAX ALLOWED QUERY : 500 CHARS ദൂരം:
നഗരങ്ങളിലെ നിങ്ങളുടെ നിർദ്ദിഷ്ട ആരംഭ, അവസാന പോയിന്റുകളെ ആശ്രയിച്ച് തൃശ്ശൂരിൽ നിന്ന് കൊച്ചിയിലേക്കുള്ള മൊത്തം ഡ്രൈവിംഗ് ദൂരം ഏകദേശം 80-90 കിലോമീറ്ററാണ്.

റോഡ് അവസ്ഥകളും ട്രാഫിക്കും വ്യത്യാസപ്പെടാമെന്നത് ശ്രദ്ധിക്കുക, അതിനാൽ നിങ്ങളുടെ യാത്രയ്ക്കിടെ തത്സമയ അപ്‌ഡേറ്റുകൾക്കായി ഒരു നാവിഗേഷൻ ആപ്പ് ഉപയോഗിക്കുന്നത് നല്ലതാണ്.
Enter your request: exit
Exiting


In [49]:
import os
import re
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM
from translate import Translator

# Set the OpenAI API key (replace with your actual key if needed)
os.environ["OPENAI_API_KEY"] = ""
# Uncomment below line if it's needed
# openai.api_key = os.getenv("OPENAI_API_KEY")

# Function to extract locations from user input
def extract_locations(user_input):
    patterns = [
        re.compile(r"take me from\s+(.+?)\s+to\s+(.+)", re.IGNORECASE),
        re.compile(r"from\s+(.+?)\s+to\s+(.+)", re.IGNORECASE),
        re.compile(r"i want to go from\s+(.+?)\s+to\s+(.+)", re.IGNORECASE)
    ]

    for pattern in patterns:
        match = pattern.search(user_input)
        if match:
            start_location = match.group(1).strip()
            end_location = match.group(2).strip()
            return start_location, end_location
    return None, None

# Function to generate route description using IndicCorp model
def generate_route_description(start_location, end_location, target_language='en'):
    # Initialize tokenizer and model for IndicCorp
    tokenizer = AutoTokenizer.from_pretrained("ai4bharat/indic-bert")
    model = AutoModelForSeq2SeqLM.from_pretrained("ai4bharat/indic-bert")

    # Prepare inputs and encode them
    inputs = f"Provide a detailed driving route from {start_location} to {end_location}. Include major turns, landmarks, and distances."
    input_ids = tokenizer.encode(inputs, return_tensors="pt", max_length=512, truncation=True)

    # Generate response from IndicCorp
    translator = Translator(to_lang=target_language)
    route_description = translator.translate(inputs)

    return route_description

# Run the chatbot in a loop
print("Greetings! I am SARATHI, developed by NIT Calicut students. Type 'exit' to quit.")
while True:
    user_input = input("Enter your request: ")
    if user_input.lower() == 'exit':
        print('Exiting')
        break

    start_location, end_location = extract_locations(user_input)
    if not start_location or not end_location:
        print("Invalid input format. Please use phrases like 'take me from [start location] to [end location]', 'from [start location] to [end location]', or 'I want to go from [start location] to [end location]'.")
        continue

    # Example: Translating to Hindi ('hi'). You can change 'hi' to any other language code as needed.
    route_description = generate_route_description(start_location, end_location, target_language='hi')

    language_name = {
        'en': 'English',
        'hi': 'Hindi'
        # Add more languages as needed
    }.get('hi', 'Unknown')

    print(f"Route Description (in {language_name}): {route_description}")


Greetings! I am SARATHI, developed by NIT Calicut students. Type 'exit' to quit.
Enter your request: from kannur to kozhikode


ValueError: Unrecognized configuration class <class 'transformers.models.albert.configuration_albert.AlbertConfig'> for this kind of AutoModel: AutoModelForSeq2SeqLM.
Model type should be one of BartConfig, BigBirdPegasusConfig, BlenderbotConfig, BlenderbotSmallConfig, EncoderDecoderConfig, FSMTConfig, GPTSanJapaneseConfig, LEDConfig, LongT5Config, M2M100Config, MarianConfig, MBartConfig, MT5Config, MvpConfig, NllbMoeConfig, PegasusConfig, PegasusXConfig, PLBartConfig, ProphetNetConfig, SeamlessM4TConfig, SeamlessM4Tv2Config, SwitchTransformersConfig, T5Config, UMT5Config, XLMProphetNetConfig.

In [50]:
import os
import re
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM

# Function to extract locations from user input
def extract_locations(user_input):
    patterns = [
        re.compile(r"take me from\s+(.+?)\s+to\s+(.+)", re.IGNORECASE),
        re.compile(r"from\s+(.+?)\s+to\s+(.+)", re.IGNORECASE),
        re.compile(r"i want to go from\s+(.+?)\s+to\s+(.+)", re.IGNORECASE)
    ]

    for pattern in patterns:
        match = pattern.search(user_input)
        if match:
            start_location = match.group(1).strip()
            end_location = match.group(2).strip()
            return start_location, end_location
    return None, None

# Function to generate route description using facebook bart multilingual
def generate_route_description(start_location, end_location, target_language='en'):
  # Initialize tokenizer and model for facebook bart multilingual
  tokenizer = AutoTokenizer.from_pretrained("facebook/bart-base-multilingual")
  model = AutoModelForSeq2SeqLM.from_pretrained("facebook/bart-base-multilingual")


  inputs = f"Provide a detailed driving route from {start_location} to {end_location} in English. Include major turns, landmarks, and distances."
  input_ids = tokenizer(inputs, return_tensors="pt", max_length=512, truncation=True)

  # Generate response from the model
  output = model.generate(**input_ids)
  route_description = tokenizer.decode(output[0], skip_special_tokens=True).strip()

  # Translate the route description if needed
  if target_language != 'en':
    translator = pipeline("translation", model="facebook/bart-base-multilingual")
    translated_route_description = translator(route_description, source_language="en", target_language=target_language)[0]["translation_text"]
    return translated_route_description
  else:
    return route_description

print("Greetings! I am SARATHI, developed by NIT Calicut students. Type 'exit' to quit.")
while True:
    user_input = input("Enter your request: ")
    if user_input.lower() == 'exit':
        print('Exiting')
        break

    start_location, end_location = extract_locations(user_input)
    if not start_location or not end_location:
        print("Invalid input format. Please use phrases like 'take me from [start location] to [end location]', 'from [start location] to [end location]', or 'I want to go from [start location] to [end location]'.")
        continue

    # Example: Translating to Hindi ('hi'). You can change 'hi' to any other language code as needed.
    route_description = generate_route_description(start_location, end_location, target_language='hi')

    language_name = {
        'en': 'English',
        'hi': 'Hindi'
        # Add more languages as needed
    }.get('hi', 'Unknown')

    print(f"Route Description (in {language_name}): {route_description}")


Greetings! I am SARATHI, developed by NIT Calicut students. Type 'exit' to quit.
Enter your request: from kannur to kozhikode


OSError: facebook/bart-base-multilingual is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`

In [52]:
import os
import re
from transformers import MarianMTModel, MarianTokenizer

# Set the OpenAI API key (replace with your actual key if needed)
os.environ["OPENAI_API_KEY"] = ""
# Uncomment below line if it's needed
# openai.api_key = os.getenv("OPENAI_API_KEY")

# Function to extract locations from user input
def extract_locations(user_input):
    patterns = [
        re.compile(r"take me from\s+(.+?)\s+to\s+(.+)", re.IGNORECASE),
        re.compile(r"from\s+(.+?)\s+to\s+(.+)", re.IGNORECASE),
        re.compile(r"i want to go from\s+(.+?)\s+to\s+(.+)", re.IGNORECASE)
    ]

    for pattern in patterns:
        match = pattern.search(user_input)
        if match:
            start_location = match.group(1).strip()
            end_location = match.group(2).strip()
            return start_location, end_location
    return None, None

# Function to generate route description using Helsinki model
def generate_route_description(start_location, end_location, target_language='en'):
    # Initialize tokenizer and model for Helsinki model (MarianMT)
    model_name = f'Helsinki-NLP/opus-mt-en-hi'
    model = MarianMTModel.from_pretrained(model_name)
    tokenizer = MarianTokenizer.from_pretrained(model_name)

    # Prepare inputs and encode them
    inputs = f"Provide a detailed driving route from {start_location} to {end_location}. Include major turns, landmarks, and distances."
    inputs = tokenizer.prepare_seq2seq_batch([inputs], return_tensors='pt')

    # Generate response from Helsinki model
    translated_ids = model.generate(**inputs)
    route_description = tokenizer.decode(translated_ids[0], skip_special_tokens=True)

    return route_description

# Run the chatbot in a loop
print("Greetings! I am SARATHI, developed by NIT Calicut students. Type 'exit' to quit.")
while True:
    user_input = input("Enter your request: ")
    if user_input.lower() == 'exit':
        print('Exiting')
        break

    start_location, end_location = extract_locations(user_input)
    if not start_location or not end_location:
        print("Invalid input format. Please use phrases like 'take me from [start location] to [end location]', 'from [start location] to [end location]', or 'I want to go from [start location] to [end location]'.")
        continue

    # Generate initial route description in English
    route_description = generate_route_description(start_location, end_location, target_language='en')
    print(f"Route Description (in English): {route_description}")

    # Ask user if they want the description in another language
    while True:
        user_choice = input("Do you want the description in another language? (y/n): ").lower()
        if user_choice == 'y':
            print("Available languages: Hindi (hi), Bengali (bn), Tamil (ta), Telugu (te), Marathi (mr), Gujarati (gu), Kannada (kn), Malayalam (ml), Punjabi (pa), Odia (or)")
            target_language = input("Enter the language code (e.g., 'hi' for Hindi): ").strip().lower()
            # Ensure valid language code and translate if needed
            if target_language in ['en', 'hi', 'bn', 'ta', 'te', 'mr', 'gu', 'kn', 'ml', 'pa', 'or']:  # Add more languages as needed
                route_description = generate_route_description(start_location, end_location, target_language=target_language)
                language_name = {
                    'en': 'English',
                    'hi': 'Hindi',
                    'bn': 'Bengali',
                    'ta': 'Tamil',
                    'te': 'Telugu',
                    'mr': 'Marathi',
                    'gu': 'Gujarati',
                    'kn': 'Kannada',
                    'ml': 'Malayalam',
                    'pa': 'Punjabi',
                    'or': 'Odia'
                    # Add more languages as needed
                }.get(target_language, 'Unknown')
                print(f"Route Description (in {language_name}): {route_description}")
                break
            else:
                print("Unsupported language. Please try again.")
        elif user_choice == 'n':
            break
        else:
            print("Invalid choice. Please enter 'y' or 'n'.")


Greetings! I am SARATHI, developed by NIT Calicut students. Type 'exit' to quit.
Enter your request: from kannur to kozhikode


config.json:   0%|          | 0.00/1.39k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/306M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

...

target.spm:   0%|          | 0.00/1.07M [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.10M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:4072: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the regular
`__call__` method to prepare your inputs and targets.

Here is a short example:

model_inputs = tokenizer(src_texts, text_target=tgt_texts, ...)

If you either need to use different keyword arguments for the source and target texts, you should do two calls like
this:

model_inputs = tokenizer(src_texts, ...)
labels = tokenizer(text_target=tgt_texts, ...)
model_inputs["labels"] = labels["input_ids"]

See the documentation of your specific tokenizer for more details on the specific arguments to the tokenizer of choice.
For a more complete example, see the implementation of `pr

Route Description (in English): कोर से कोर तक एक विस्तृत गाड़ी मार्ग प्रदान करें ।
Do you want the description in another language? (y/n): y
Available languages: Hindi (hi), Bengali (bn), Tamil (ta), Telugu (te), Marathi (mr), Gujarati (gu), Kannada (kn), Malayalam (ml), Punjabi (pa), Odia (or)
Enter the language code (e.g., 'hi' for Hindi): hi
Route Description (in Hindi): कोर से कोर तक एक विस्तृत गाड़ी मार्ग प्रदान करें ।


KeyboardInterrupt: Interrupted by user

In [42]:
import os
import re
import openai

# Set the OpenAI API key (replace with your actual key)
os.environ["OPENAI_API_KEY"] = ""
openai.api_key = os.getenv("OPENAI_API_KEY")

def extract_locations(user_input):
    # Regular expressions to extract locations from input
    patterns = [
        re.compile(r"take me from\s+(.+?)\s+to\s+(.+)", re.IGNORECASE),
        re.compile(r"from\s+(.+?)\s+to\s+(.+)", re.IGNORECASE),
        re.compile(r"i want to go from\s+(.+?)\s+to\s+(.+)", re.IGNORECASE)
    ]

    for pattern in patterns:
        match = pattern.search(user_input)
        if match:
            start_location = match.group(1).strip()
            end_location = match.group(2).strip()
            return start_location, end_location
    return None, None

def generate_route_description(start_location, end_location, target_language='en'):
    # Prompt for the OpenAI API
    prompt = f"Provide a detailed driving route from {start_location} to {end_location}. Include major turns, landmarks, and distances."

    # Generate response from OpenAI using AI-4 India's multilingual model
    response = openai.Completion.create(
        model="gpt-3.5-turbo",  # AI-4 India's multilingual model
        prompt=prompt,
        max_tokens=1000,  # Adjust as needed
        stop=["\n"]
    )
    route_description = response.choices[0].text.strip()

    return route_description

# Function to ask user for language choice
def ask_language():
    print("Please choose the language for the route description:")
    print("1. English")
    print("2. Malayalam")
    # Add more languages as needed

    while True:
        choice = input("Enter your choice (1/2): ").strip()
        if choice == '1':
            return 'en'  # English
        elif choice == '2':
            return 'ml'  # Malayalam
        else:
            print("Invalid choice. Please enter 1 or 2.")

# Run the chatbot in a loop
print("Greetings! I am SARATHI, developed by NIT Calicut students. Type 'exit' to quit.")
while True:
    user_input = input("Enter your request: ")
    if user_input.lower() == 'exit':
        print('Exiting')
        break

    start_location, end_location = extract_locations(user_input)
    if not start_location or not end_location:
        print("Invalid input format. Please use phrases like 'take me from [start location] to [end location]', 'from [start location] to [end location]', or 'I want to go from [start location] to [end location]'.")
        continue

    # Ask user for language choice
    target_language = ask_language()

    # Generate route description based on chosen language
    route_description = generate_route_description(start_location, end_location, target_language)

    language_name = {
        'en': 'English',
        'ml': 'Malayalam'
        # Add more languages as needed
    }.get(target_language, 'Unknown')

    print(f"Route Description (in {language_name}):")
    print(route_description)


Greetings! I am SARATHI, developed by NIT Calicut students. Type 'exit' to quit.
Enter your request: from thrissur to kochi
Please choose the language for the route description:
1. English
2. Malayalam
Enter your choice (1/2): 2


InvalidRequestError: This is a chat model and not supported in the v1/completions endpoint. Did you mean to use v1/chat/completions?

In [78]:
!pip install googletrans==4.0.0-rc1

In [80]:
import os
import openai
import re
from googletrans import Translator

# Set the OpenAI API key (replace with your actual key)
os.environ["OPENAI_API_KEY"] = ""
openai.api_key = os.getenv("OPENAI_API_KEY")

translator = Translator()

def detect_language(text):
    response = openai.Completion.create(
        engine="gpt-3.5-turbo-instruct",
        prompt=f"Detect the language of the following text:\n\n{text}",
        max_tokens=5
    )
    return response.choices[0].text.strip()

def translate_to_english(text, source_language):
    try:
        translation = translator.translate(text, src=source_language, dest='en')
        return translation.text
    except Exception as e:
        print(f"Error translating to English: {e}")
        return text

def extract_locations(user_input):
    patterns = [
        re.compile(r"take me from\s+(.+?)\s+to\s+(.+)", re.IGNORECASE),
        re.compile(r"from\s+(.+?)\s+to\s+(.+)", re.IGNORECASE),
        re.compile(r"i want to go from\s+(.+?)\s+to\s+(.+)", re.IGNORECASE)
    ]

    for pattern in patterns:
        match = pattern.search(user_input)
        if match:
            start_location = match.group(1).strip()
            end_location = match.group(2).strip()
            return start_location, end_location
    return None, None

def generate_route_description(start_location, end_location):
    prompt = f"Provide a detailed driving route from {start_location} to {end_location}. Include major turns, landmarks, and distances."

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ]
    )

    return response.choices[0].message['content'].strip()

def translate_description(route_description, target_language):
    if target_language == 'en':
        return route_description
    else:
        try:
            translation = translator.translate(route_description, dest=target_language)
            return translation.text
        except Exception as e:
            print(f"Error translating description: {e}")
            return route_description

print("Greetings! I am SARATHI, developed by NIT Calicut students. Type 'exit' to quit.")
while True:
    user_input = input("Enter your request: ")
    if user_input.lower() == 'exit':
        print('Exiting')
        break

    detected_language = detect_language(user_input)
    if detected_language.lower() in ['hindi', 'malayalam']:
        user_input = translate_to_english(user_input, detected_language.lower())

    start_location, end_location = extract_locations(user_input)
    if not start_location or not end_location:
        print("Invalid input format. Please use phrases like 'take me from [start location] to [end location]', 'from [start location] to [end location]', or 'I want to go from [start location] to [end location]'.")
        continue

    route_description = generate_route_description(start_location, end_location)
    print("Route Description (in English):", route_description)

    target_language = input("Enter language code for translation (e.g., 'hi' for Hindi, 'ml' for Malayalam): ").lower()

    translated_description = translate_description(route_description, target_language)
    print(f"Route Description (in {target_language}):", translated_description)


Greetings! I am SARATHI, developed by NIT Calicut students. Type 'exit' to quit.
Enter your request: തൃശൂർ മുതൽ കൊച്ചി വരെ
Invalid input format. Please use phrases like 'take me from [start location] to [end location]', 'from [start location] to [end location]', or 'I want to go from [start location] to [end location]'.


KeyboardInterrupt: Interrupted by user

In [44]:
import os
import re
import openai

# Set the OpenAI API key (replace with your actual key)
os.environ["OPENAI_API_KEY"] = ""
openai.api_key = os.getenv("OPENAI_API_KEY")

def extract_locations(user_input):
    # Regular expressions to extract locations from input
    patterns = [
        re.compile(r"take me from\s+(.+?)\s+to\s+(.+)", re.IGNORECASE),
        re.compile(r"from\s+(.+?)\s+to\s+(.+)", re.IGNORECASE),
        re.compile(r"i want to go from\s+(.+?)\s+to\s+(.+)", re.IGNORECASE)
    ]

    for pattern in patterns:
        match = pattern.search(user_input)
        if match:
            start_location = match.group(1).strip()
            end_location = match.group(2).strip()
            return start_location, end_location
    return None, None

def generate_route_description(start_location, end_location, target_language='en'):
    # Prompt for the OpenAI API
    prompt = f"Provide a detailed driving route from {start_location} to {end_location}. Include major turns, landmarks, and distances."

    # Generate response from OpenAI using AI-4 India's multilingual model
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-instruct",  # AI-4 India's multilingual model
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=500,  # Adjust as needed
        stop=["\n"]
    )
    route_description = response.choices[0].message['content'].strip()

    return route_description

# Function to ask user for language choice
def ask_language():
    print("Please choose the language for the route description:")
    print("1. English")
    print("2. Malayalam")
    # Add more languages as needed

    while True:
        choice = input("Enter your choice (1/2): ").strip()
        if choice == '1':
            return 'en'  # English
        elif choice == '2':
            return 'ml'  # Malayalam
        else:
            print("Invalid choice. Please enter 1 or 2.")

# Run the chatbot in a loop
print("Greetings! I am SARATHI, developed by NIT Calicut students. Type 'exit' to quit.")
while True:
    user_input = input("Enter your request: ")
    if user_input.lower() == 'exit':
        print('Exiting')
        break

    start_location, end_location = extract_locations(user_input)
    if not start_location or not end_location:
        print("Invalid input format. Please use phrases like 'take me from [start location] to [end location]', 'from [start location] to [end location]', or 'I want to go from [start location] to [end location]'.")
        continue

    # Ask user for language choice
    target_language = ask_language()

    # Generate route description based on chosen language
    route_description = generate_route_description(start_location, end_location, target_language)

    language_name = {
        'en': 'English',
        'ml': 'Malayalam'
        # Add more languages as needed
    }.get(target_language, 'Unknown')

    print(f"Route Description (in {language_name}):")
    print(route_description)


Greetings! I am SARATHI, developed by NIT Calicut students. Type 'exit' to quit.
Enter your request: from kannur to kozhikode
Please choose the language for the route description:
1. English
2. Malayalam
Enter your choice (1/2): 2


InvalidRequestError: This is not a chat model and thus not supported in the v1/chat/completions endpoint. Did you mean to use v1/completions?

In [45]:
import os
import re
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM

# Set the OpenAI API key (replace with your actual key if needed)
os.environ["OPENAI_API_KEY"] =
# Function to extract locations from user input
def extract_locations(user_input):
    patterns = [
        re.compile(r"take me from\s+(.+?)\s+to\s+(.+)", re.IGNORECASE),
        re.compile(r"from\s+(.+?)\s+to\s+(.+)", re.IGNORECASE),
        re.compile(r"i want to go from\s+(.+?)\s+to\s+(.+)", re.IGNORECASE)
    ]

    for pattern in patterns:
        match = pattern.search(user_input)
        if match:
            start_location = match.group(1).strip()
            end_location = match.group(2).strip()
            return start_location, end_location
    return None, None

# Function to generate route description using Indic-BERT
def generate_route_description(start_location, end_location, target_language='en'):
    # Initialize tokenizer and model for Indic-BERT
    tokenizer = AutoTokenizer.from_pretrained("ai4bharat/indic-bert")
    model = AutoModelForSeq2SeqLM.from_pretrained("ai4bharat/indic-bert")

    # Prepare inputs and encode them
    inputs = f"Provide a detailed driving route from {start_location} to {end_location}. Include major turns, landmarks, and distances."
    input_ids = tokenizer.encode(inputs, return_tensors="pt", max_length=512, truncation=True)

    # Generate response from Indic-BERT
    translated_texts = pipeline(
        "translation_xx_to_yy",  # Replace with the appropriate translation model
        model=model,
        tokenizer=tokenizer
    )
    route_description = translated_texts(inputs, target_lang=target_language)[0]['translation_text']

    return route_description

# Function to ask user for language choice
def ask_language():
    print("Please choose the language for the route description:")
    print("1. English")
    print("2. Malayalam")
    # Add more languages as needed

    while True:
        choice = input("Enter your choice (1/2): ").strip()
        if choice == '1':
            return 'en'  # English
        elif choice == '2':
            return 'ml'  # Malayalam
        else:
            print("Invalid choice. Please enter 1 or 2.")

# Run the chatbot in a loop
print("Greetings! I am SARATHI, developed by NIT Calicut students. Type 'exit' to quit.")
while True:
    user_input = input("Enter your request: ")
    if user_input.lower() == 'exit':
        print('Exiting')
        break

    start_location, end_location = extract_locations(user_input)
    if not start_location or not end_location:
        print("Invalid input format. Please use phrases like 'take me from [start location] to [end location]', 'from [start location] to [end location]', or 'I want to go from [start location] to [end location]'.")
        continue

    # Ask user for language choice
    target_language = ask_language()

    # Generate route description based on chosen language using Indic-BERT
    route_description = generate_route_description(start_location, end_location, target_language)

    language_name = {
        'en': 'English',
        'ml': 'Malayalam'
        # Add more languages as needed
    }.get(target_language, 'Unknown')

    print(f"Route Description (in {language_name}):")
    print(route_description)


Greetings! I am SARATHI, developed by NIT Calicut students. Type 'exit' to quit.
Enter your request: from kannur to kozhikode
Please choose the language for the route description:
1. English
2. Malayalam
Enter your choice (1/2): 2


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/507 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/5.65M [00:00<?, ?B/s]

ValueError: Unrecognized configuration class <class 'transformers.models.albert.configuration_albert.AlbertConfig'> for this kind of AutoModel: AutoModelForSeq2SeqLM.
Model type should be one of BartConfig, BigBirdPegasusConfig, BlenderbotConfig, BlenderbotSmallConfig, EncoderDecoderConfig, FSMTConfig, GPTSanJapaneseConfig, LEDConfig, LongT5Config, M2M100Config, MarianConfig, MBartConfig, MT5Config, MvpConfig, NllbMoeConfig, PegasusConfig, PegasusXConfig, PLBartConfig, ProphetNetConfig, SeamlessM4TConfig, SeamlessM4Tv2Config, SwitchTransformersConfig, T5Config, UMT5Config, XLMProphetNetConfig.

In [47]:
import os
import re
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM

# Set the OpenAI API key (replace with your actual key if needed)
os.environ["OPENAI_API_KEY"] = ""

# Function to extract locations from user input
def extract_locations(user_input):
    patterns = [
        re.compile(r"take me from\s+(.+?)\s+to\s+(.+)", re.IGNORECASE),
        re.compile(r"from\s+(.+?)\s+to\s+(.+)", re.IGNORECASE),
        re.compile(r"i want to go from\s+(.+?)\s+to\s+(.+)", re.IGNORECASE)
    ]

    for pattern in patterns:
        match = pattern.search(user_input)
        if match:
            start_location = match.group(1).strip()
            end_location = match.group(2).strip()
            return start_location, end_location
    return None, None

# Function to generate route description using Indic-BERT
def generate_route_description(start_location, end_location, target_language='en'):
    # Initialize tokenizer and model for Indic-BERT
    tokenizer = AutoTokenizer.from_pretrained("ai4bharat/indic-bert")
    model = AutoModelForSeq2SeqLM.from_pretrained("ai4bharat/indic-bert")

    # Prepare inputs and encode them
    inputs = f"Provide a detailed driving route from {start_location} to {end_location}. Include major turns, landmarks, and distances."
    input_ids = tokenizer.encode(inputs, return_tensors="pt", max_length=512, truncation=True)

    # Generate response from Indic-BERT
    translated_texts = pipeline(
        "translation_xx_to_yy",  # Replace with the appropriate translation model
        model=model,
        tokenizer=tokenizer
    )
    route_description = translated_texts(inputs, target_lang=target_language)[0]['translation_text']

    return route_description

# Function to ask user for language choice
def ask_language():
    print("Please choose the language for the route description:")
    print("1. English")
    print("2. Malayalam")
    # Add more languages as needed

    while True:
        choice = input("Enter your choice (1/2): ").strip()
        if choice == '1':
            return 'en'  # English
        elif choice == '2':
            return 'ml'  # Malayalam
        else:
            print("Invalid choice. Please enter 1 or 2.")

# Run the chatbot in a loop
print("Greetings! I am SARATHI, developed by NIT Calicut students. Type 'exit' to quit.")
while True:
    user_input = input("Enter your request: ")
    if user_input.lower() == 'exit':
        print('Exiting')
        break

    start_location, end_location = extract_locations(user_input)
    if not start_location or not end_location:
        print("Invalid input format. Please use phrases like 'take me from [start location] to [end location]', 'from [start location] to [end location]', or 'I want to go from [start location] to [end location]'.")
        continue

    # Ask user for language choice
    target_language = ask_language()

    # Generate route description based on chosen language using Indic-BERT
    route_description = generate_route_description(start_location, end_location, target_language)

    language_name = {
        'en': 'English',
        'ml': 'Malayalam'
        # Add more languages as needed
    }.get(target_language, 'Unknown')

    print(f"Route Description (in {language_name}):")
    print(route_description)


Greetings! I am SARATHI, developed by NIT Calicut students. Type 'exit' to quit.
Enter your request: from kannur to kozhikode
Please choose the language for the route description:
1. English
2. Malayalam
Enter your choice (1/2): 1


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


ValueError: Unrecognized configuration class <class 'transformers.models.albert.configuration_albert.AlbertConfig'> for this kind of AutoModel: AutoModelForSeq2SeqLM.
Model type should be one of BartConfig, BigBirdPegasusConfig, BlenderbotConfig, BlenderbotSmallConfig, EncoderDecoderConfig, FSMTConfig, GPTSanJapaneseConfig, LEDConfig, LongT5Config, M2M100Config, MarianConfig, MBartConfig, MT5Config, MvpConfig, NllbMoeConfig, PegasusConfig, PegasusXConfig, PLBartConfig, ProphetNetConfig, SeamlessM4TConfig, SeamlessM4Tv2Config, SwitchTransformersConfig, T5Config, UMT5Config, XLMProphetNetConfig.

In [31]:
import os
import re
from transformers import AutoModelForCausalLM, AutoTokenizer
from translate import Translator

# Set the Hugging Face model and tokenizer for multilingual support
model_name = "microsoft/xtremedistil-multilingual-17"
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

def extract_locations(user_input):
    # Regular expressions to extract locations from input
    patterns = [
        re.compile(r"take me from\s+(.+?)\s+to\s+(.+)", re.IGNORECASE),
        re.compile(r"from\s+(.+?)\s+to\s+(.+)", re.IGNORECASE),
        re.compile(r"i want to go from\s+(.+?)\s+to\s+(.+)", re.IGNORECASE)
    ]

    for pattern in patterns:
        match = pattern.search(user_input)
        if match:
            start_location = match.group(1).strip()
            end_location = match.group(2).strip()
            return start_location, end_location
    return None, None

def generate_route_description(start_location, end_location, target_language='en'):
    # Prompt for the model
    prompt = f"Provide a detailed driving route from {start_location} to {end_location}. Include major turns, landmarks, and distances."

    # Tokenize prompt
    inputs = tokenizer(prompt, return_tensors="pt")

    # Generate response
    with torch.no_grad():
        outputs = model.generate(inputs.input_ids, max_length=150)

    # Decode response
    route_description = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Translate the route description to the target language
    translator = Translator(to_lang=target_language)
    translated_description = translator.translate(route_description)

    return translated_description

# Run the chatbot in a loop
print("Greetings! I am SARATHI, developed by NIT Calicut students. Type 'exit' to quit.")
while True:
    user_input = input("Enter your request: ")
    if user_input.lower() == 'exit':
        print('Exiting')
        break

    start_location, end_location = extract_locations(user_input)
    if not start_location or not end_location:
        print("Invalid input format. Please use phrases like 'take me from [start location] to [end location]', 'from [start location] to [end location]', or 'I want to go from [start location] to [end location]'.")
        continue

    # Example: Translating to Spanish ('es'). You can change 'es' to any other language code as needed.
    route_description = generate_route_description(start_location, end_location, target_language='es')
    print("Route Description (in Spanish):", route_description)


OSError: microsoft/xtremedistil-multilingual-17 is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`

In [22]:
import os
import openai
import re

# Set the OpenAI API key (replace with your actual key)
os.environ["OPENAI_API_KEY"] = ""
openai.api_key = os.getenv("OPENAI_API_KEY")

def extract_locations(user_input):
  # Regular expressions to extract locations (adapt for other languages)
  patterns = [
      re.compile(r"(?:take me from|from|I want to go from)\s+(.+?)\s+to\s+(.+)", re.IGNORECASE),
  ]

  for pattern in patterns:
    match = pattern.search(user_input)
    if match:
      start_location = match.group(1).strip()
      end_location = match.group(2).strip()
      return start_location, end_location
  return None, None

def generate_route_description(start_location, end_location, language="en"):
  # Prompt for the OpenAI API with language parameter
  prompt = f"Provide a detailed driving route from {start_location} to {end_location} in {language}. Include major turns, landmarks, and distances."

  response = openai.ChatCompletion.create(
      model="gpt-3.5-turbo",
      messages=[
          {"role": "system", "content": "You are a helpful assistant."},
          {"role": "user", "content": prompt}
      ]
  )

  return response.choices[0].message.content.strip()

# Run the chatbot in a loop
print("Greetings! I am SARATHI, developed by NIT Calicut students. Type 'exit' to quit.")
while True:
  user_input = input("Enter your request: ")
  if user_input.lower() == 'exit':
    print('Exiting')
    break

  start_location, end_location = extract_locations(user_input)
  if not start_location or not end_location:
    print("Invalid input format. Please use phrases like 'take me from [start location] to [end location]', 'from [start location] to [end location]', or 'I want to go from [start location] to [end location]'.")
    continue

  # Get user language preference
  language = input("Choose your preferred language (en for English, otherwise specify): ")

  route_description = generate_route_description(start_location, end_location, language)
  print("Route Description:", route_description)


Greetings! I am SARATHI, developed by NIT Calicut students. Type 'exit' to quit.
Enter your request: take me from kannur to kozhikode
Choose your preferred language (en for English, otherwise specify): hi
Route Description: Sure! Here is a detailed driving route from Kannur to Kozhikode:

1. Start from Kannur and head south on NH 66 (also known as Kannur - Kozhikode Road).
2. Continue on NH 66 for about 49 km.
3. You will pass through places like Thalassery and Mahe along the way.
4. After approximately 49 km, you will reach Kozhikode district.
5. Stay on NH 66 which becomes Mavoor Road as you approach Kozhikode city.
6. Once you enter Kozhikode city, continue straight on Mavoor Road.
7. Drive for about 8 km on Mavoor Road until you reach the heart of Kozhikode city.
8. Look out for major landmarks such as Mananchira Square, Sarovaram Biopark, and Kozhikode Beach as you navigate the city streets.
9. Finally, reach your destination in Kozhikode.

Distance: The total distance between Kan

In [29]:
import transformers
from transformers import pipeline
import openai
import os
import re

# Set your OpenAI API key (replace with your actual key)
os.environ["OPENAI_API_KEY"] = ""
openai.api_key = os.getenv("OPENAI_API_KEY")

# Load translation model, specifying source and target languages
translator = pipeline("translation_en_to_hi", engine="openai", api_key=os.getenv("OPENAI_API_KEY"))

def extract_locations(user_input):
  # Regular expressions to extract locations (adapt for other languages)
  patterns = [
      re.compile(r"(?:take me from|from|I want to go from)\s+(.+?)\s+to\s+(.+)", re.IGNORECASE),
  ]

  for pattern in patterns:
    match = pattern.search(user_input)
    if match:
      start_location = match.group(1).strip()
      end_location = match.group(2).strip()
      return start_location, end_location
  return None, None

def generate_route_description(start_location, end_location, language="en"):
  # Use an open-source LLM like BLOOM for route generation in English
  # Install required libraries: transformers huggingface-hub
  # pip install transformers huggingface-hub
  model_name = "bigscience/bloom"  # Replace with your chosen model
  tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)
  model = transformers.AutoModelForSeq2SeqLM.from_pretrained(model_name)

  # Prompt for route generation (ensure compatibility with your LLM)
  prompt = f"Provide a detailed driving route from {start_location} to {end_location} in English. Include major turns, landmarks, and distances."
  input_ids = tokenizer(prompt, return_tensors="pt")["input_ids"]

  # Generate route description
  output = model.generate(**input_ids)
  route_description = tokenizer.decode(output[0], skip_special_tokens=True).strip()

  # Translate the route description to the desired language (if not English)
  if language != "en":
    translated_route_description = translator(route_description, source_language="en", target_language=language)[0]["translation_text"]
    return translated_route_description
  else:
    return route_description

# Run the chatbot in a loop
print("Greetings! I am SARATHI, developed by NIT Calicut students. Type 'exit' to quit.")
while True:
  user_input = input("Enter your request: ")
  if user_input.lower() == 'exit':
    print('Exiting')
    break

  start_location, end_location = extract_locations(user_input)
  if not start_location or not end_location:
    print("Invalid input format. Please use phrases like 'take me from [start location] to [end location]', 'from [start location] to [end location]', or 'I want to go from [start location] to [end location]'.")
    continue

  # Get user language preference
  language = input("Choose your preferred language (en for English, otherwise specify): ")

  route_description = generate_route_description(start_location, end_location, language)
  print("Route Description:", route_description)


ValueError: The task does not provide any default models for options ('en', 'hi')

In [24]:
!pip install transformers
!pip install huggingface-hub

In [6]:
!pip install gradio


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 37.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 10.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.1/318.1 kB 28.8 MB/s eta 0:00:00
  Using cached httpx-0.27.0-py3-none-any.whl (75 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 kB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 58.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 16.3 MB/s eta 0:00:00
  Using cached httpcore-1.0.5-py3-none-any.whl (77 kB)
  Using cached h11-0.14.0-py3-none-any.whl (58 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 26.3 MB/s

In [18]:
import os
import openai
import re
import gradio as gr

# Set the OpenAI API key (replace with your actual key)
os.environ["OPENAI_API_KEY"] = ""
openai.api_key = os.getenv("OPENAI_API_KEY")

def extract_locations(user_input):
    # Regular expressions to extract locations from input
    patterns = [
        re.compile(r"take me from\s+(.+?)\s+to\s+(.+)", re.IGNORECASE),
        re.compile(r"from\s+(.+?)\s+to\s+(.+)", re.IGNORECASE),
        re.compile(r"i want to go from\s+(.+?)\s+to\s+(.+)", re.IGNORECASE)
    ]

    for pattern in patterns:
        match = pattern.search(user_input)
        if match:
            start_location = match.group(1).strip()
            end_location = match.group(2).strip()
            return start_location, end_location
    return None, None

def generate_route_description(start_location, end_location):
    # Prompt for the OpenAI API
    prompt = f"Provide a detailed driving route from {start_location} to {end_location}. Include major turns, landmarks, and distances."

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ]
    )

    return response.choices[0].message.content.strip()

def get_route_description(user_input):
    start_location, end_location = extract_locations(user_input)
    if not start_location or not end_location:
        return "Invalid input format. Please use phrases like 'take me from [start location] to [end location]', 'from [start location] to [end location]', or 'I want to go from [start location] to [end location]'."

    route_description = generate_route_description(start_location, end_location)
    return f"Route Description: {route_description}"

iface = gr.Interface(
    fn=get_route_description,
    inputs="text",
    outputs="text",
    title="SARATHI Route Description Chatbot",
    description="Greetings! I am SARATHI, developed by NIT Calicut students. Enter a request like 'take me from [start location] to [end location]', 'from [start location] to [end location]', or 'I want to go from [start location] to [end location]'."
)

iface.launch(debug=True)


IMPORTANT: You are using gradio version 3.1.4, however version 4.29.0 is available, please upgrade.
--------
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



*** Failed to connect to ec2.gradio.app:22: [Errno 110] Connection timed out
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/gradio/tunneling.py", line 74, in create_tunnel
    client.connect(
  File "/usr/local/lib/python3.10/dist-packages/paramiko/client.py", line 386, in connect
    sock.connect(addr)
TimeoutError: [Errno 110] Connection timed out

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-18-d72743313916>", line 56, in <cell line: 56>
    iface.launch(debug=True)
  File "/usr/local/lib/python3.10/dist-packages/gradio/blocks.py", line 895, in launch
    share_url = networking.setup_tunnel(self.server_port, None)
  File "/usr/local/lib/python3.10/dist-packages/gradio/networking.py", line 176, in setup_tunnel


TypeError: object of type 'NoneType' has no len()

In [21]:
import os
import openai
import re
import gradio as gr

# Set the OpenAI API key (replace with your actual key)
os.environ["OPENAI_API_KEY"] = ""
openai.api_key = os.getenv("OPENAI_API_KEY")

def extract_locations(user_input):
    # Regular expressions to extract locations from input
    patterns = [
        re.compile(r"take me from\s+(.+?)\s+to\s+(.+)", re.IGNORECASE),
        re.compile(r"from\s+(.+?)\s+to\s+(.+)", re.IGNORECASE),
        re.compile(r"i want to go from\s+(.+?)\s+to\s+(.+)", re.IGNORECASE)
    ]

    for pattern in patterns:
        match = pattern.search(user_input)
        if match:
            start_location = match.group(1).strip()
            end_location = match.group(2).strip()
            return start_location, end_location
    return None, None

def generate_route_description(start_location, end_location):
    # Prompt for the OpenAI API
    prompt = f"Provide a detailed driving route from {start_location} to {end_location}. Include major turns, landmarks, and distances."

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ]
    )

    return response.choices[0].message.content.strip()

def get_route_description(user_input):
    start_location, end_location = extract_locations(user_input)
    if not start_location or not end_location:
        return "Invalid input format. Please use phrases like 'take me from [start location] to [end location]', 'from [start location] to [end location]', or 'I want to go from [start location] to [end location]'."

    route_description = generate_route_description(start_location, end_location)
    return f"Route Description: {route_description}"

# Define Gradio interface
iface = gr.Interface(
    fn=get_route_description,
    inputs=gr.inputs.Textbox(label="Enter your request:"),
    outputs=gr.outputs.Textbox(label="Route Description"),
    title="SARATHI Route Description Chatbot",
    description="Greetings! I am SARATHI, developed by NIT Calicut students. Enter a request like 'take me from [start location] to [end location]', 'from [start location] to [end location]', or 'I want to go from [start location] to [end location]'."
)

# Launch Gradio app in Colab
iface.launch(debug=True,share=True)


/usr/local/lib/python3.10/dist-packages/gradio/inputs.py:26: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/gradio/deprecation.py:40: UserWarning: `optional` parameter is deprecated, and it has no effect
  warnings.warn(value)
/usr/local/lib/python3.10/dist-packages/gradio/deprecation.py:40: UserWarning: `numeric` parameter is deprecated, and it has no effect
  warnings.warn(value)
/usr/local/lib/python3.10/dist-packages/gradio/deprecation.py:40: UserWarning: The 'type' parameter has been deprecated. Use the Number component instead.
  warnings.warn(value)
/usr/local/lib/python3.10/dist-packages/gradio/outputs.py:21: UserWarning: Usage of gradio.outputs is deprecated, and will not be supported in the future, please import your components from gradio.components
  warnings.warn(


IMPORTANT: You are using gradio version 3.1.4, however version 4.29.0 is available, please upgrade.
--------
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



*** Failed to connect to ec2.gradio.app:22: [Errno 110] Connection timed out
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/gradio/tunneling.py", line 74, in create_tunnel
    client.connect(
  File "/usr/local/lib/python3.10/dist-packages/paramiko/client.py", line 386, in connect
    sock.connect(addr)
TimeoutError: [Errno 110] Connection timed out

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-21-3f29a7d58ba9>", line 58, in <cell line: 58>
    iface.launch(debug=True,share=True)
  File "/usr/local/lib/python3.10/dist-packages/gradio/blocks.py", line 895, in launch
    share_url = networking.setup_tunnel(self.server_port, None)
  File "/usr/local/lib/python3.10/dist-packages/gradio/networking.py", line 176, in se

TypeError: object of type 'NoneType' has no len()

In [6]:
import os
import openai
import re
import gradio as gr

# Set the OpenAI API key (replace with your actual key)
os.environ["OPENAI_API_KEY"] = ""
openai.api_key = os.getenv("OPENAI_API_KEY")

def extract_locations(user_input):
    # Regular expressions to extract locations from input
    patterns = [
        re.compile(r"take me from\s+(.+?)\s+to\s+(.+)", re.IGNORECASE),
        re.compile(r"from\s+(.+?)\s+to\s+(.+)", re.IGNORECASE),
        re.compile(r"i want to go from\s+(.+?)\s+to\s+(.+)", re.IGNORECASE)
    ]

    for pattern in patterns:
        match = pattern.search(user_input)
        if match:
            start_location = match.group(1).strip()
            end_location = match.group(2).strip()
            return start_location, end_location
    return None, None

def generate_route_description(start_location, end_location):
    # Prompt for the OpenAI API
    prompt = f"Provide a detailed driving route from {start_location} to {end_location}. Include major turns, landmarks, and distances."

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ]
    )

    return response.choices[0].message.content.strip()

def get_route_description(user_input):
    start_location, end_location = extract_locations(user_input)
    if not start_location or not end_location:
        return "Invalid input format. Please use phrases like 'take me from [start location] to [end location]', 'from [start location] to [end location]', or 'I want to go from [start location] to [end location]'."

    route_description = generate_route_description(start_location, end_location)
    return f"Route Description: {route_description}"

# Define Gradio interface
iface = gr.Interface(
    fn=get_route_description,
    inputs="text",
    outputs="text",
    title="SARATHI Route Description Chatbot",
    description="Greetings! I am SARATHI, developed by NIT Calicut students. Enter a request like 'take me from [start location] to [end location]', 'from [start location] to [end location]', or 'I want to go from [start location] to [end location]'."
)

# Launch Gradio interface in Colab
iface.launch(inline=True)


IMPORTANT: You are using gradio version 3.1.4, however version 4.29.0 is available, please upgrade.
--------
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



*** Failed to connect to ec2.gradio.app:22: [Errno 110] Connection timed out
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/gradio/tunneling.py", line 74, in create_tunnel
    client.connect(
  File "/usr/local/lib/python3.10/dist-packages/paramiko/client.py", line 386, in connect
    sock.connect(addr)
TimeoutError: [Errno 110] Connection timed out

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-6-60c141473cb0>", line 58, in <cell line: 58>
    iface.launch(inline=True)
  File "/usr/local/lib/python3.10/dist-packages/gradio/blocks.py", line 895, in launch
    share_url = networking.setup_tunnel(self.server_port, None)
  File "/usr/local/lib/python3.10/dist-packages/gradio/networking.py", line 176, in setup_tunnel


TypeError: object of type 'NoneType' has no len()

In [10]:
import os
import openai
import re
import streamlit as st

# Set the OpenAI API key (replace with your actual key)
os.environ["OPENAI_API_KEY"] = ""
openai.api_key = os.getenv("OPENAI_API_KEY")

def extract_locations(user_input):
    # Regular expressions to extract locations from input
    patterns = [
        re.compile(r"take me from\s+(.+?)\s+to\s+(.+)", re.IGNORECASE),
        re.compile(r"from\s+(.+?)\s+to\s+(.+)", re.IGNORECASE),
        re.compile(r"i want to go from\s+(.+?)\s+to\s+(.+)", re.IGNORECASE)
    ]

    for pattern in patterns:
        match = pattern.search(user_input)
        if match:
            start_location = match.group(1).strip()
            end_location = match.group(2).strip()
            return start_location, end_location
    return None, None

def generate_route_description(start_location, end_location):
    # Prompt for the OpenAI API
    prompt = f"Provide a detailed driving route from {start_location} to {end_location}. Include major turns, landmarks, and distances."

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ]
    )

    return response.choices[0].message.content.strip()

def get_route_description(user_input):
    start_location, end_location = extract_locations(user_input)
    if not start_location or not end_location:
        return "Invalid input format. Please use phrases like 'take me from [start location] to [end location]', 'from [start location] to [end location]', or 'I want to go from [start location] to [end location]'."

    route_description = generate_route_description(start_location, end_location)
    return f"Route Description: {route_description}"

# Streamlit web app
def main():
    st.title("SARATHI Route Description Chatbot")
    st.markdown("Greetings! I am SARATHI, developed by NIT Calicut students. Enter a request like 'take me from [start location] to [end location]', 'from [start location] to [end location]', or 'I want to go from [start location] to [end location]'.")

    user_input = st.text_input("Enter your request:")
    if user_input:
        route_description = get_route_description(user_input)
        st.text(route_description)

if __name__ == "__main__":
    main()


In [9]:
import os
import openai
import re
import streamlit as st

# Set the OpenAI API key (replace with your actual key)
os.environ["OPENAI_API_KEY"] = ""
openai.api_key = os.getenv("OPENAI_API_KEY")

def extract_locations(user_input):
    # Regular expressions to extract locations from input
    patterns = [
        re.compile(r"take me from\s+(.+?)\s+to\s+(.+)", re.IGNORECASE),
        re.compile(r"from\s+(.+?)\s+to\s+(.+)", re.IGNORECASE),
        re.compile(r"i want to go from\s+(.+?)\s+to\s+(.+)", re.IGNORECASE)
    ]

    for pattern in patterns:
        match = pattern.search(user_input)
        if match:
            start_location = match.group(1).strip()
            end_location = match.group(2).strip()
            return start_location, end_location
    return None, None

def generate_route_description(start_location, end_location):
    # Prompt for the OpenAI API
    prompt = f"Provide a detailed driving route from {start_location} to {end_location}. Include major turns, landmarks, and distances."

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ]
    )

    return response.choices[0].message.content.strip()

def get_route_description(user_input):
    start_location, end_location = extract_locations(user_input)
    if not start_location or not end_location:
        return "Invalid input format. Please use phrases like 'take me from [start location] to [end location]', 'from [start location] to [end location]', or 'I want to go from [start location] to [end location]'."

    route_description = generate_route_description(start_location, end_location)
    return f"Route Description: {route_description}"

# Streamlit web app
def main():
    st.title("SARATHI Route Description Chatbot")
    st.markdown("Greetings! I am SARATHI, developed by NIT Calicut students. Enter a request like 'take me from [start location] to [end location]', 'from [start location] to [end location]', or 'I want to go from [start location] to [end location]'.")

    user_input = st.text_input("Enter your request:")
    if user_input:
        route_description = get_route_description(user_input)
        st.text(route_description)

if __name__ == "__main__":
    main()


In [11]:
!pip install streamlit pyngrok


In [12]:
from pyngrok import ngrok
import streamlit as st
import os
import openai
import re
import streamlit as st

# Set the OpenAI API key (replace with your actual key)
os.environ["OPENAI_API_KEY"] = ""
openai.api_key = os.getenv("OPENAI_API_KEY")

def extract_locations(user_input):
    # Regular expressions to extract locations from input
    patterns = [
        re.compile(r"take me from\s+(.+?)\s+to\s+(.+)", re.IGNORECASE),
        re.compile(r"from\s+(.+?)\s+to\s+(.+)", re.IGNORECASE),
        re.compile(r"i want to go from\s+(.+?)\s+to\s+(.+)", re.IGNORECASE)
    ]

    for pattern in patterns:
        match = pattern.search(user_input)
        if match:
            start_location = match.group(1).strip()
            end_location = match.group(2).strip()
            return start_location, end_location
    return None, None

def generate_route_description(start_location, end_location):
    # Prompt for the OpenAI API
    prompt = f"Provide a detailed driving route from {start_location} to {end_location}. Include major turns, landmarks, and distances."

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ]
    )

    return response.choices[0].message.content.strip()

def get_route_description(user_input):
    start_location, end_location = extract_locations(user_input)
    if not start_location or not end_location:
        return "Invalid input format. Please use phrases like 'take me from [start location] to [end location]', 'from [start location] to [end location]', or 'I want to go from [start location] to [end location]'."

    route_description = generate_route_description(start_location, end_location)
    return f"Route Description: {route_description}"

# Streamlit web app
def main():
    st.title("SARATHI Route Description Chatbot")
    st.markdown("Greetings! I am SARATHI, developed by NIT Calicut students. Enter a request like 'take me from [start location] to [end location]', 'from [start location] to [end location]', or 'I want to go from [start location] to [end location]'.")

    user_input = st.text_input("Enter your request:")
    if user_input:
        route_description = get_route_description(user_input)
        st.text(route_description)

if __name__ == "__main__":
    main()



public_url = ngrok.connect(port='8501')
print('Streamlit app can be accessed at:', public_url)


ERROR:pyngrok.process.ngrok:t=2024-06-20T04:54:45+0000 lvl=eror msg="failed to reconnect session" obj=tunnels.session err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n"
ERROR:pyngrok.process.ngrok:t=2024-06-20T04:54:45+0000 lvl=eror msg="session closing" obj=tunnels.session err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n"


PyngrokNgrokError: The ngrok process errored on start: authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n.

In [15]:
from pyngrok import ngrok

ngrok.set_auth_token('')


In [16]:
import os
import openai
import re
import streamlit as st
from pyngrok import ngrok

# Set the OpenAI API key (replace with your actual key)
os.environ["OPENAI_API_KEY"] = ""
openai.api_key = os.getenv("OPENAI_API_KEY")

def extract_locations(user_input):
    # Regular expressions to extract locations from input
    patterns = [
        re.compile(r"take me from\s+(.+?)\s+to\s+(.+)", re.IGNORECASE),
        re.compile(r"from\s+(.+?)\s+to\s+(.+)", re.IGNORECASE),
        re.compile(r"i want to go from\s+(.+?)\s+to\s+(.+)", re.IGNORECASE)
    ]

    for pattern in patterns:
        match = pattern.search(user_input)
        if match:
            start_location = match.group(1).strip()
            end_location = match.group(2).strip()
            return start_location, end_location
    return None, None

def generate_route_description(start_location, end_location):
    # Prompt for the OpenAI API
    prompt = f"Provide a detailed driving route from {start_location} to {end_location}. Include major turns, landmarks, and distances."

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ]
    )

    return response.choices[0].message.content.strip()

def get_route_description(user_input):
    start_location, end_location = extract_locations(user_input)
    if not start_location or not end_location:
        return "Invalid input format. Please use phrases like 'take me from [start location] to [end location]', 'from [start location] to [end location]', or 'I want to go from [start location] to [end location]'."

    route_description = generate_route_description(start_location, end_location)
    return f"Route Description: {route_description}"

# Streamlit web app
def main():
    st.title("SARATHI Route Description Chatbot")
    st.markdown("Greetings! I am SARATHI, developed by NIT Calicut students. Enter a request like 'take me from [start location] to [end location]', 'from [start location] to [end location]', or 'I want to go from [start location] to [end location]'.")

    user_input = st.text_input("Enter your request:")
    if user_input:
        route_description = get_route_description(user_input)
        st.text(route_description)

if __name__ == "__main__":
    # Set ngrok auth token
    ngrok.set_auth_token('your_ngrok_authtoken')

    # Launch Streamlit app with ngrok
    public_url = ngrok.connect(port='8501')
    print('Streamlit app can be accessed at:', public_url)

    # Prevent the script from terminating while using ngrok
    try:
        main()
    except KeyboardInterrupt:
        pass


ERROR:pyngrok.process.ngrok:t=2024-06-20T04:59:29+0000 lvl=eror msg="failed to reconnect session" obj=tunnels.session err="authentication failed: The authtoken you specified is an ngrok v1 authtoken, but you're using ngrok v2.\nYour authtoken: your_ngrok_authtoken\nInstructions to install your authtoken are on your ngrok dashboard:\nhttps://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_106\r\n"
ERROR:pyngrok.process.ngrok:t=2024-06-20T04:59:29+0000 lvl=eror msg="session closing" obj=tunnels.session err="authentication failed: The authtoken you specified is an ngrok v1 authtoken, but you're using ngrok v2.\nYour authtoken: your_ngrok_authtoken\nInstructions to install your authtoken are on your ngrok dashboard:\nhttps://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_106\r\n"
ERROR:pyngrok.process.ngrok:t=2024-06-20T04:59:29+0000 lvl=eror msg="terminating with error" obj=app err="authentication failed: The authtoken you specified is an ngrok v1 authtoke

PyngrokNgrokError: The ngrok process errored on start: authentication failed: The authtoken you specified is an ngrok v1 authtoken, but you're using ngrok v2.\nYour authtoken: your_ngrok_authtoken\nInstructions to install your authtoken are on your ngrok dashboard:\nhttps://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_106\r\n.

In [17]:
import os
import openai
import re
import gradio as gr

# Set the OpenAI API key (replace with your actual key)
os.environ["OPENAI_API_KEY"] = ""
openai.api_key = os.getenv("OPENAI_API_KEY")

def extract_locations(user_input):
    # Regular expressions to extract locations from input
    patterns = [
        re.compile(r"take me from\s+(.+?)\s+to\s+(.+)", re.IGNORECASE),
        re.compile(r"from\s+(.+?)\s+to\s+(.+)", re.IGNORECASE),
        re.compile(r"i want to go from\s+(.+?)\s+to\s+(.+)", re.IGNORECASE)
    ]

    for pattern in patterns:
        match = pattern.search(user_input)
        if match:
            start_location = match.group(1).strip()
            end_location = match.group(2).strip()
            return start_location, end_location
    return None, None

def generate_route_description(start_location, end_location):
    # Prompt for the OpenAI API
    prompt = f"Provide a detailed driving route from {start_location} to {end_location}. Include major turns, landmarks, and distances."

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ]
    )

    return response.choices[0].message.content.strip()

def get_route_description(user_input):
    start_location, end_location = extract_locations(user_input)
    if not start_location or not end_location:
        return "Invalid input format. Please use phrases like 'take me from [start location] to [end location]', 'from [start location] to [end location]', or 'I want to go from [start location] to [end location]'."

    route_description = generate_route_description(start_location, end_location)
    return f"Route Description: {route_description}"

# Define Gradio interface
iface = gr.Interface(
    fn=get_route_description,
    inputs=gr.inputs.Textbox(label="Enter your request:"),
    outputs=gr.outputs.Textbox(label="Route Description"),
    title="SARATHI Route Description Chatbot",
    description="Greetings! I am SARATHI, developed by NIT Calicut students. Enter a request like 'take me from [start location] to [end location]', 'from [start location] to [end location]', or 'I want to go from [start location] to [end location]'."
)

# Launch Gradio app
iface.launch()


/usr/local/lib/python3.10/dist-packages/gradio/inputs.py:26: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/gradio/deprecation.py:40: UserWarning: `optional` parameter is deprecated, and it has no effect
  warnings.warn(value)
/usr/local/lib/python3.10/dist-packages/gradio/deprecation.py:40: UserWarning: `numeric` parameter is deprecated, and it has no effect
  warnings.warn(value)
/usr/local/lib/python3.10/dist-packages/gradio/deprecation.py:40: UserWarning: The 'type' parameter has been deprecated. Use the Number component instead.
  warnings.warn(value)
/usr/local/lib/python3.10/dist-packages/gradio/outputs.py:21: UserWarning: Usage of gradio.outputs is deprecated, and will not be supported in the future, please import your components from gradio.components
  warnings.warn(


IMPORTANT: You are using gradio version 3.1.4, however version 4.29.0 is available, please upgrade.
--------
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



*** Failed to connect to ec2.gradio.app:22: [Errno 110] Connection timed out
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/gradio/tunneling.py", line 74, in create_tunnel
    client.connect(
  File "/usr/local/lib/python3.10/dist-packages/paramiko/client.py", line 386, in connect
    sock.connect(addr)
TimeoutError: [Errno 110] Connection timed out

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-17-acee51e84d91>", line 58, in <cell line: 58>
    iface.launch()
  File "/usr/local/lib/python3.10/dist-packages/gradio/blocks.py", line 895, in launch
    share_url = networking.setup_tunnel(self.server_port, None)
  File "/usr/local/lib/python3.10/dist-packages/gradio/networking.py", line 176, in setup_tunnel
    return

TypeError: object of type 'NoneType' has no len()

In [1]:
!pip install gradio==3.1.4


In [2]:
!pip install openai==0.28
!pip install googletrans

  Using cached httpx-0.13.3-py3-none-any.whl (55 kB)
  Using cached httpcore-0.9.1-py3-none-any.whl (42 kB)
  Using cached h11-0.9.0-py2.py3-none-any.whl (53 kB)
  Attempting uninstall: h11
    Found existing installation: h11 0.12.0
    Uninstalling h11-0.12.0:
      Successfully uninstalled h11-0.12.0
  Attempting uninstall: httpcore
    Found existing installation: httpcore 0.15.0
    Uninstalling httpcore-0.15.0:
      Successfully uninstalled httpcore-0.15.0
  Attempting uninstall: httpx
    Found existing installation: httpx 0.25.1
    Uninstalling httpx-0.25.1:
      Successfully uninstalled httpx-0.25.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
fastapi 0.111.0 requires httpx>=0.23.0, but you have httpx 0.13.3 which is incompatible.
gradio 3.1.4 requires h11<0.13,>=0.11, but you have h11 0.9.0 which is incompatible.
gradio-client 1.0.1 requires htt

In [3]:
!pip install ffmpeg-python

In [81]:
import pandas as pd

# Load the dataset
file_path = '/content/cleaned_first_100_issues.csv'
data = pd.read_csv(file_path)

# Display the first few rows of the dataset
print(data.head())


                                        title  \
0  Added python code example for parking lot    
1                            Optimised Images   
2                         de: Add translation   
3                               small changes   
4                 ru: Add Russian translation   

                                              labels  \
0  [{"name:": "needs-review", "description:": "no...   
1  [{"name:": "needs-review", "description:": "no...   
2  [{"name:": "needs-review", "description:": "no...   
3  [{"name:": "needs-review", "description:": "no...   
4  [{"name:": "needs-review", "description:": "no...   

                                                body  \
0  …sign\n\n## Review the Contributing Guidelines...   
1  ## Review the Contributing Guidelines\n\nBefor...   
2  ## Review the Contributing Guidelines\n\nBefor...   
3  ## Review the Contributing Guidelines\n\nBefor...   
4  * Added README-ru.md\n* Kept content of origin...   

                               

In [86]:
from langchain.chains import LLMChain
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate

# Initialize the language model
llm = OpenAI()

# Create a prompt template for issue handling
prompt_template = PromptTemplate(
    input_variables=["issue_title", "issue_body", "issue_labels", "issue_url", "repository_url", "language"],
    template=(
        "Issue Title: {issue_title}\n"
        "Issue Body: {issue_body}\n"
        "Issue Labels: {issue_labels}\n"
        "Issue URL: {issue_url}\n"
        "Repository URL: {repository_url}\n"
        "Language: {language}\n"
        "Response:"
    )
)

# Create a chain for issue handling
issue_handling_chain = LLMChain(llm=llm, prompt=prompt_template)

issue_title = data['title'][0]
issue_body = data['body'][0]
issue_labels = data['labels'][0]
issue_url = data['issue_url'][0]
repository_url = data['repository_url'][0]
language = data['language'][0]

response = issue_handling_chain.run({
    "issue_title": issue_title,
    "issue_body": issue_body,
    "issue_labels": issue_labels,
    "issue_url": issue_url,
    "repository_url": repository_url,
    "language": language
})
print(response)


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(



Title: Added python code example for parking lot
Body: ...sign

## Review the Contributing Guidelines

Before submitting a pull request, verify it meets all requirements in the [Contributing Guidelines](https://github.com/donnemartin/system-design-primer/blob/master/CONTRIBUTING.md).

### Translations

See the [Contributing Guidelines](https://github.com/donnemartin/system-design-primer/blob/master/CONTRIBUTING.md). Verify you've:

* Tagged the [language maintainer](https://github.com/donnemartin/system-design-primer/blob/master/TRANSLATIONS.md)
* Prefixed the title with a language code
    * Example: "ja: Fix ..."

Issue Labels: [{"name": "needs-review", "description": "none", "color": "5319e7"}]
Issue URL: https://api.github.com/repos/donnemartin/system-design-primer/issues/507
Repository URL: https://api.github.com/repos/donnemartin/system-design-primer
Language: en
Response: Success


In [ ]:
from langchain.chains import LLMChain
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate

# Initialize the language model
llm = OpenAI()

# Create a prompt template for issue handling
prompt_template = PromptTemplate(
    input_variables=["issue_title", "issue_body", "issue_labels", "issue_url", "repository_url", "language"],
    template=(
        "Issue Title: {issue_title}\n"
        "Issue Body: {issue_body}\n"
        "Issue Labels: {issue_labels}\n"
        "Issue URL: {issue_url}\n"
        "Repository URL: {repository_url}\n"
        "Language: {language}\n"
        "Response:"
    )
)

# Create a chain for issue handling
issue_handling_chain = LLMChain(llm=llm, prompt=prompt_template)

# Interactive loop for handling issues
while True:
    print("\nPlease enter the details of the issue (or type 'exit' to quit):")

    # Collect user input
    issue_title = input("Issue Title: ")
    if issue_title.lower() == 'exit':
        break

    issue_body = input("Issue Body: ")
    issue_labels = input("Issue Labels (comma-separated): ")
    issue_url = input("Issue URL: ")
    repository_url = input("Repository URL: ")
    language = input("Language: ")

    # Prepare input dictionary for the chain
    input_data = {
        "issue_title": issue_title,
        "issue_body": issue_body,
        "issue_labels": issue_labels,
        "issue_url": issue_url,
        "repository_url": repository_url,
        "language": language
    }

    # Run the issue handling chain
    response = issue_handling_chain.run(input_data)

    # Print the response
    print("\nResponse:")
    print(response)


In [ ]:
from flask import Flask, request, jsonify
from langchain.chains import ConversationChain
from langchain.memory import ChatMessageHistory
from langchain.llms import OpenAI

app = Flask(__name__)

# Initialize the language model and conversation chain
llm = OpenAI()
conversation_history = ChatMessageHistory()
conversation_chain = ConversationChain(llm=llm, memory=conversation_history)

@app.route('/chat', methods=['POST'])
def chat():
    user_input = request.json.get('message')
    response = conversation_chain.run(user_input)
    return jsonify({'response': response})

@app.route('/issue', methods=['POST'])
def handle_issue():
    issue_title = request.json.get('issue_title')
    issue_body = request.json.get('issue_body')
    issue_labels = request.json.get('issue_labels')
    issue_url = request.json.get('issue_url')
    repository_url = request.json.get('repository_url')
    language = request.json.get('language')

    response = issue_handling_chain.run({
        "issue_title": issue_title,
        "issue_body": issue_body,
        "issue_labels": issue_labels,
        "issue_url": issue_url,
        "repository_url": repository_url,
        "language": language
    })
    return jsonify({'response': response})

if __name__ == '__main__':
    app.run(debug=True)


In [85]:
!pip install langchain
!pip install langchain_community
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 6.4 MB/s eta 0:00:00
